In [ ]:
#Prophet

In [1]:
!pip install prophet

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from prophet import Prophet
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import warnings
import gc

warnings.filterwarnings("ignore")

# ======================================================
# Paths
# ======================================================
input_folder = "dataset"
output_models = "prophet_output_models_weekly"
output_csv = "prophet_output_csv_weekly"
output_graphs = "prophet_output_graphs_weekly"
metrics_file = "prophet_metrics_weekly.csv"

os.makedirs(output_models, exist_ok=True)
os.makedirs(output_csv, exist_ok=True)
os.makedirs(output_graphs, exist_ok=True)

# ======================================================
# Helper Functions
# ======================================================
def parse_dates_safe(date_series):
    try:
        return pd.to_datetime(date_series, dayfirst=True)
    except:
        return pd.to_datetime(date_series, dayfirst=False)

def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    mask = y_true != 0
    return np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100

# ======================================================
# Process Each Crop File
# ======================================================
metrics_list = []

for file in sorted(os.listdir(input_folder)):
    if not file.endswith(".csv"):
        continue

    print(f"\n🚀 Processing: {file}")
    file_path = os.path.join(input_folder, file)

    # -------------------------------------
    # Load Data
    # -------------------------------------
    df = pd.read_csv(file_path)
    df['Date'] = parse_dates_safe(df['Date'])
    df = df.sort_values('Date')
    df = df.groupby('Date', as_index=False).mean(numeric_only=True)

    df.set_index('Date', inplace=True)
    df = df.asfreq('W')
    df['Average Price'] = df['Average Price'].ffill().bfill()

    # -------------------------------------
    # Feature Engineering (Weekly)
    # -------------------------------------
    df['Lag_1'] = df['Average Price'].shift(1)
    df['Lag_4'] = df['Average Price'].shift(4)
    df['MA_4'] = df['Average Price'].rolling(window=4).mean()
    df['MA_12'] = df['Average Price'].rolling(window=12).mean()
    df = df.bfill()

    # Prophet expects ds, y columns
    df_prophet = df.reset_index().rename(columns={'Date': 'ds', 'Average Price': 'y'})

    # Add custom regressors
    df_prophet['Lag_1'] = df_prophet['Lag_1']
    df_prophet['Lag_4'] = df_prophet['Lag_4']
    df_prophet['MA_4'] = df_prophet['MA_4']
    df_prophet['MA_12'] = df_prophet['MA_12']

    # -------------------------------------
    # Build Prophet Model with Regressors
    # -------------------------------------
    m = Prophet(
        yearly_seasonality=True,
        weekly_seasonality=False,
        daily_seasonality=False,
        seasonality_mode='additive',
        changepoint_prior_scale=0.5
    )

    m.add_regressor('Lag_1')
    m.add_regressor('Lag_4')
    m.add_regressor('MA_4')
    m.add_regressor('MA_12')

    # Train model
    m.fit(df_prophet)

    # -------------------------------------
    # Forecasting (Full + Future 12 weeks)
    # -------------------------------------
    future = m.make_future_dataframe(periods=12, freq='W')
    future = future.merge(df_prophet[['ds', 'Lag_1', 'Lag_4', 'MA_4', 'MA_12']], on='ds', how='left')

    # Fill missing future regressor values using last known
    for col in ['Lag_1', 'Lag_4', 'MA_4', 'MA_12']:
        future[col] = future[col].ffill().bfill()

    forecast = m.predict(future)

    # -------------------------------------
    # Evaluation (on known data only)
    # -------------------------------------
    valid_mask = df_prophet['y'].notna()
    y_true = df_prophet.loc[valid_mask, 'y']
    y_pred = forecast.loc[:len(df_prophet)-1, 'yhat']

    mae = round(mean_absolute_error(y_true, y_pred), 2)
    rmse = round(np.sqrt(mean_squared_error(y_true, y_pred)), 2)
    r2 = round(r2_score(y_true, y_pred), 4)
    mape = round(mean_absolute_percentage_error(y_true, y_pred), 2)
    acc = round(100 - mape, 2)

    metrics_list.append({
        'Crop': file.replace('.csv',''),
        'MAE': mae,
        'RMSE': rmse,
        'R2': r2,
        'MAPE(%)': mape,
        'Accuracy(%)': acc
    })

    print(f"✅ {file}: MAE={mae}, RMSE={rmse}, R²={r2}, MAPE={mape}%, Accuracy={acc}%")

    # -------------------------------------
    # Save Outputs
    # -------------------------------------
    forecast_merged = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]
    forecast_merged = forecast_merged.rename(columns={
        'ds': 'Date', 'yhat': 'Predicted', 'yhat_lower': 'Lower_Bound', 'yhat_upper': 'Upper_Bound'
    })
    merged_df = df_prophet.merge(forecast_merged, left_on='ds', right_on='Date', how='left')
    merged_df = merged_df[['Date', 'y', 'Predicted', 'Lower_Bound', 'Upper_Bound',
                           'Lag_1', 'Lag_4', 'MA_4', 'MA_12']]
    merged_df = merged_df.rename(columns={'y': 'Actual'})
    merged_df = merged_df.round(2)

    # Save updated CSV
    merged_df.to_csv(os.path.join(output_csv, file.replace('.csv','_prophet_weekly_updated.csv')), index=False)

    # -------------------------------------
    # Save Graph
    # -------------------------------------
    plt.figure(figsize=(12,6))
    plt.plot(merged_df['Date'], merged_df['Actual'], label='Actual', color='blue')
    plt.plot(merged_df['Date'], merged_df['Predicted'], label='Predicted', color='red', linestyle='dashed')
    plt.fill_between(merged_df['Date'], merged_df['Lower_Bound'], merged_df['Upper_Bound'],
                     color='lightgray', alpha=0.3, label='Confidence Interval')
    plt.title(f"Prophet Weekly Forecast - {file}")
    plt.xlabel("Date")
    plt.ylabel("Average Price")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(os.path.join(output_graphs, file.replace('.csv','_prophet_weekly_graph.png')))
    plt.close()

    # Save model
    model_path = os.path.join(output_models, file.replace('.csv','_prophet_model.pkl'))
    with open(model_path, 'wb') as f:
        import pickle
        pickle.dump(m, f)

    gc.collect()

# ======================================================
# Save Final Metrics
# ======================================================
metrics_df = pd.DataFrame(metrics_list).round(2)
metrics_df.to_csv(metrics_file, index=False)
print("\n📊 All Prophet weekly metrics saved successfully!")
print(metrics_df)


🚀 Processing: capsicum_2010_2024_master_weekly.csv


22:18:54 - cmdstanpy - INFO - Chain [1] start processing
22:18:55 - cmdstanpy - INFO - Chain [1] done processing


✅ capsicum_2010_2024_master_weekly.csv: MAE=100.03, RMSE=137.56, R²=0.9644, MAPE=3.95%, Accuracy=96.05%

🚀 Processing: onion_2010_2024_master_weekly.csv


22:18:56 - cmdstanpy - INFO - Chain [1] start processing
22:18:57 - cmdstanpy - INFO - Chain [1] done processing


✅ onion_2010_2024_master_weekly.csv: MAE=41.11, RMSE=68.63, R²=0.9896, MAPE=2.31%, Accuracy=97.69%

🚀 Processing: tomato_2010_2024_master_weekly.csv


22:18:57 - cmdstanpy - INFO - Chain [1] start processing
22:18:58 - cmdstanpy - INFO - Chain [1] done processing


✅ tomato_2010_2024_master_weekly.csv: MAE=82.5, RMSE=136.61, R²=0.9685, MAPE=6.16%, Accuracy=93.84%

🚀 Processing: wheat_2010_2024_master_weekly.csv


22:18:59 - cmdstanpy - INFO - Chain [1] start processing
22:18:59 - cmdstanpy - INFO - Chain [1] done processing


✅ wheat_2010_2024_master_weekly.csv: MAE=13.84, RMSE=22.69, R²=0.998, MAPE=0.62%, Accuracy=99.38%

📊 All Prophet weekly metrics saved successfully!
                               Crop     MAE    RMSE    R2  MAPE(%)  \
0  capsicum_2010_2024_master_weekly  100.03  137.56  0.96     3.95   
1     onion_2010_2024_master_weekly   41.11   68.63  0.99     2.31   
2    tomato_2010_2024_master_weekly   82.50  136.61  0.97     6.16   
3     wheat_2010_2024_master_weekly   13.84   22.69  1.00     0.62   

   Accuracy(%)  
0        96.05  
1        97.69  
2        93.84  
3        99.38  


In [ ]:
#ARIMA

In [3]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gc
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import warnings

warnings.filterwarnings("ignore")

# ======================================================
# Paths
# ======================================================
input_folder = "dataset"
output_models = "arima_output_models_weekly"
output_csv = "arima_output_csv_weekly"
output_graphs = "arima_output_graphs_weekly"
metrics_file = "arima_metrics_weekly.csv"

os.makedirs(output_models, exist_ok=True)
os.makedirs(output_csv, exist_ok=True)
os.makedirs(output_graphs, exist_ok=True)

# ======================================================
# Helper Functions
# ======================================================
def parse_dates_safe(date_series):
    try:
        return pd.to_datetime(date_series, dayfirst=True)
    except:
        return pd.to_datetime(date_series, dayfirst=False)

def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    mask = y_true != 0
    return np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100

# ======================================================
# Metrics Storage
# ======================================================
metrics_list = []

# ======================================================
# Process Each CSV File
# ======================================================
for file in sorted(os.listdir(input_folder)):
    if not file.endswith(".csv"):
        continue

    print(f"\n🚀 Processing Crop File: {file}")
    file_path = os.path.join(input_folder, file)
    df = pd.read_csv(file_path)

    # --------------------------------------------
    # Parse and prepare
    # --------------------------------------------
    df['Date'] = parse_dates_safe(df['Date'])
    df = df.sort_values('Date')
    df = df.groupby('Date', as_index=False).mean(numeric_only=True)
    df.set_index('Date', inplace=True)

    # Weekly frequency alignment
    df = df.asfreq('W')
    df['Average Price'] = df['Average Price'].ffill().bfill()

    # --------------------------------------------
    # Feature Engineering (Weekly)
    # --------------------------------------------
    df['Lag_1'] = df['Average Price'].shift(1)
    df['Lag_4'] = df['Average Price'].shift(4)
    df['MA_4'] = df['Average Price'].rolling(window=4).mean()
    df['MA_12'] = df['Average Price'].rolling(window=12).mean()
    df = df.bfill()

    # --------------------------------------------
    # Model Training
    # --------------------------------------------
    y = df['Average Price']

    # Simple ARIMA (p,d,q)
    order = (1, 1, 1)

    try:
        model = ARIMA(y, order=order)
        model_fit = model.fit()
    except Exception as e:
        print(f"❌ ARIMA model failed for {file}: {e}")
        continue

    # --------------------------------------------
    # Predictions
    # --------------------------------------------
    df['Predicted'] = model_fit.predict(start=0, end=len(df)-1)

    # --------------------------------------------
    # Metrics
    # --------------------------------------------
    y_true = df['Average Price'].values
    y_pred = df['Predicted'].values

    mae = round(mean_absolute_error(y_true, y_pred), 2)
    rmse = round(np.sqrt(mean_squared_error(y_true, y_pred)), 2)
    r2 = round(r2_score(y_true, y_pred), 4)
    mape = round(mean_absolute_percentage_error(y_true, y_pred), 2)
    acc = round(100 - mape, 2)

    print(f"✅ {file} | MAE={mae}, RMSE={rmse}, R²={r2}, MAPE={mape}%, Accuracy={acc}%")

    metrics_list.append({
        'Crop': file.replace('.csv',''),
        'MAE': mae,
        'RMSE': rmse,
        'R2': r2,
        'MAPE(%)': mape,
        'Accuracy(%)': acc
    })

    # --------------------------------------------
    # Save updated CSV
    # --------------------------------------------
    df = df.round(2)
    updated_csv_path = os.path.join(output_csv, file.replace('.csv', '_arima_weekly_updated.csv'))
    df.to_csv(updated_csv_path)

    # --------------------------------------------
    # Save model parameters
    # --------------------------------------------
    model_file = os.path.join(output_models, file.replace('.csv', '_arima_model.txt'))
    with open(model_file, 'w') as f:
        f.write(str(model_fit.summary()))

    # --------------------------------------------
    # Plot Graph
    # --------------------------------------------
    plt.figure(figsize=(12,6))
    plt.plot(df.index, df['Average Price'], label='Actual', color='blue')
    plt.plot(df.index, df['Predicted'], label='Predicted (ARIMA)', color='red', linestyle='dashed')
    plt.plot(df.index, df['MA_4'], label='MA 4', color='green', linestyle='--')
    plt.plot(df.index, df['MA_12'], label='MA 12', color='orange', linestyle='--')
    plt.title(f"ARIMA Weekly Forecast - {file}")
    plt.xlabel("Date")
    plt.ylabel("Average Price")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(os.path.join(output_graphs, file.replace('.csv', '_arima_weekly_graph.png')))
    plt.close()

    gc.collect()

# ======================================================
# Save Final Metrics
# ======================================================
metrics_df = pd.DataFrame(metrics_list).round(2)
metrics_df.to_csv(metrics_file, index=False)
print("\n📊 All ARIMA Weekly metrics saved successfully!")
print(metrics_df)


🚀 Processing Crop File: capsicum_2010_2024_master_weekly.csv
✅ capsicum_2010_2024_master_weekly.csv | MAE=104.9, RMSE=156.5, R²=0.9539, MAPE=4.11%, Accuracy=95.89%

🚀 Processing Crop File: onion_2010_2024_master_weekly.csv
✅ onion_2010_2024_master_weekly.csv | MAE=44.9, RMSE=94.89, R²=0.9801, MAPE=2.41%, Accuracy=97.59%

🚀 Processing Crop File: tomato_2010_2024_master_weekly.csv
✅ tomato_2010_2024_master_weekly.csv | MAE=84.63, RMSE=158.4, R²=0.9577, MAPE=5.88%, Accuracy=94.12%

🚀 Processing Crop File: wheat_2010_2024_master_weekly.csv
✅ wheat_2010_2024_master_weekly.csv | MAE=17.71, RMSE=66.47, R²=0.9827, MAPE=0.82%, Accuracy=99.18%

📊 All ARIMA Weekly metrics saved successfully!
                               Crop     MAE    RMSE    R2  MAPE(%)  \
0  capsicum_2010_2024_master_weekly  104.90  156.50  0.95     4.11   
1     onion_2010_2024_master_weekly   44.90   94.89  0.98     2.41   
2    tomato_2010_2024_master_weekly   84.63  158.40  0.96     5.88   
3     wheat_2010_2024_master_

In [ ]:
#SARIMAX

In [4]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gc
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import warnings
import joblib

warnings.filterwarnings("ignore")

# ======================================================
# Paths
# ======================================================
input_folder = "dataset"
output_models = "sarimax_output_models_weekly"
output_csv = "sarimax_output_csv_weekly"
output_graphs = "sarimax_output_graphs_weekly"
metrics_file = "sarimax_metrics_weekly.csv"

os.makedirs(output_models, exist_ok=True)
os.makedirs(output_csv, exist_ok=True)
os.makedirs(output_graphs, exist_ok=True)

# ======================================================
# Helper Functions
# ======================================================
def parse_dates_safe(date_series):
    """Safe date parsing supporting both DD-MM-YYYY and MM-DD-YYYY."""
    try:
        return pd.to_datetime(date_series, dayfirst=True)
    except:
        return pd.to_datetime(date_series, dayfirst=False)

def mean_absolute_percentage_error(y_true, y_pred):
    """Compute MAPE with zero protection."""
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    mask = y_true != 0
    return np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100

# ======================================================
# Metrics Storage
# ======================================================
metrics_list = []

# ======================================================
# Process Each Crop File
# ======================================================
for file in sorted(os.listdir(input_folder)):
    if not file.endswith(".csv"):
        continue

    print(f"\n🚀 Processing Crop File: {file}")
    file_path = os.path.join(input_folder, file)

    # --------------------------------------------
    # Load and Clean Data
    # --------------------------------------------
    df = pd.read_csv(file_path)
    df['Date'] = parse_dates_safe(df['Date'])
    df = df.sort_values('Date')
    df = df.groupby('Date', as_index=False).mean(numeric_only=True)
    df.set_index('Date', inplace=True)

    # Ensure weekly frequency
    df = df.asfreq('W')
    df['Average Price'] = df['Average Price'].ffill().bfill()

    # --------------------------------------------
    # Feature Engineering (Weekly)
    # --------------------------------------------
    df['Lag_1'] = df['Average Price'].shift(1)
    df['Lag_4'] = df['Average Price'].shift(4)
    df['MA_4'] = df['Average Price'].rolling(window=4).mean()
    df['MA_12'] = df['Average Price'].rolling(window=12).mean()
    df = df.bfill().ffill()

    # Exogenous features for SARIMAX
    exog_features = df[['Lag_1', 'Lag_4', 'MA_4', 'MA_12']]

    # --------------------------------------------
    # Train SARIMAX Model
    # --------------------------------------------
    order = (1, 1, 1)           # (p, d, q)
    seasonal_order = (1, 1, 1, 52)  # Weekly seasonality (52 weeks per year)

    try:
        model = SARIMAX(df['Average Price'],
                        exog=exog_features,
                        order=order,
                        seasonal_order=seasonal_order,
                        enforce_stationarity=False,
                        enforce_invertibility=False)
        model_fit = model.fit(disp=False)
    except Exception as e:
        print(f"❌ Model failed for {file}: {e}")
        continue

    # --------------------------------------------
    # Predictions
    # --------------------------------------------
    df['Predicted'] = model_fit.predict(start=0, end=len(df)-1, exog=exog_features)

    # --------------------------------------------
    # Metrics Calculation
    # --------------------------------------------
    y_true = df['Average Price']
    y_pred = df['Predicted']
    mae = round(mean_absolute_error(y_true, y_pred), 2)
    rmse = round(np.sqrt(mean_squared_error(y_true, y_pred)), 2)
    r2 = round(r2_score(y_true, y_pred), 4)
    mape = round(mean_absolute_percentage_error(y_true, y_pred), 2)
    acc = round(100 - mape, 2)

    metrics_list.append({
        'Crop': file.replace('.csv',''),
        'MAE': mae,
        'RMSE': rmse,
        'R2': r2,
        'MAPE(%)': mape,
        'Accuracy(%)': acc
    })

    print(f"✅ {file} | MAE={mae}, RMSE={rmse}, R²={r2}, MAPE={mape}%, Accuracy={acc}%")

    # --------------------------------------------
    # Save Model
    # --------------------------------------------
    model_file = os.path.join(output_models, file.replace(".csv", "_sarimax_weekly_model.pkl"))
    joblib.dump(model_fit, model_file)

    # --------------------------------------------
    # Save Updated CSV
    # --------------------------------------------
    df = df.round(2)
    updated_csv_path = os.path.join(output_csv, file.replace(".csv", "_sarimax_weekly_updated.csv"))
    df.to_csv(updated_csv_path)

    # --------------------------------------------
    # Plot Graph
    # --------------------------------------------
    plt.figure(figsize=(12,6))
    plt.plot(df.index, df['Average Price'], label='Actual Price', color='blue')
    plt.plot(df.index, df['Predicted'], label='Predicted (SARIMAX)', color='red', linestyle='dashed')
    plt.plot(df.index, df['MA_4'], label='MA 4', color='green', linestyle='--')
    plt.plot(df.index, df['MA_12'], label='MA 12', color='orange', linestyle='--')
    plt.title(f"SARIMAX Weekly Forecast - {file}")
    plt.xlabel("Date")
    plt.ylabel("Average Price")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(os.path.join(output_graphs, file.replace(".csv", "_sarimax_weekly_graph.png")))
    plt.close()

    gc.collect()

# ======================================================
# Save Final Metrics
# ======================================================
metrics_df = pd.DataFrame(metrics_list).round(2)
metrics_df.to_csv(metrics_file, index=False)
print("\n📊 All SARIMAX Weekly Metrics Saved Successfully!")
print(metrics_df)


🚀 Processing Crop File: capsicum_2010_2024_master_weekly.csv
✅ capsicum_2010_2024_master_weekly.csv | MAE=86.28, RMSE=163.5, R²=0.9497, MAPE=3.78%, Accuracy=96.22%

🚀 Processing Crop File: onion_2010_2024_master_weekly.csv
✅ onion_2010_2024_master_weekly.csv | MAE=39.97, RMSE=124.44, R²=0.9657, MAPE=2.51%, Accuracy=97.49%

🚀 Processing Crop File: tomato_2010_2024_master_weekly.csv
✅ tomato_2010_2024_master_weekly.csv | MAE=69.21, RMSE=120.68, R²=0.9755, MAPE=5.65%, Accuracy=94.35%

🚀 Processing Crop File: wheat_2010_2024_master_weekly.csv
✅ wheat_2010_2024_master_weekly.csv | MAE=25.05, RMSE=193.36, R²=0.8533, MAPE=1.41%, Accuracy=98.59%

📊 All SARIMAX Weekly Metrics Saved Successfully!
                               Crop    MAE    RMSE    R2  MAPE(%)  Accuracy(%)
0  capsicum_2010_2024_master_weekly  86.28  163.50  0.95     3.78        96.22
1     onion_2010_2024_master_weekly  39.97  124.44  0.97     2.51        97.49
2    tomato_2010_2024_master_weekly  69.21  120.68  0.98     5.65 

In [ ]:
#TAT+MHA

In [5]:
import os
import gc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers, callbacks

# -----------------------------
# Paths / Output dirs
# -----------------------------
input_folder = "dataset"
output_models = "tat_mha_output_models_weekly"
output_csv = "tat_mha_output_csv_weekly"
output_graphs = "tat_mha_output_graphs_weekly"
metrics_file = "tat_mha_metrics_weekly.csv"

os.makedirs(output_models, exist_ok=True)
os.makedirs(output_csv, exist_ok=True)
os.makedirs(output_graphs, exist_ok=True)

# -----------------------------
# Config - change if needed
# -----------------------------
look_back = 30         # number of weeks used as input sequence
batch_size = 32
epochs = 60
val_split = 0.15
d_model = 64
num_heads = 4
ff_dim = 128
dropout_rate = 0.15
seed = 42

tf.random.set_seed(seed)
np.random.seed(seed)

# -----------------------------
# Helpers
# -----------------------------
def parse_dates_safe(s):
    try:
        return pd.to_datetime(s, dayfirst=True)
    except:
        return pd.to_datetime(s, dayfirst=False)

def mean_absolute_percentage_error(y_true, y_pred):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    mask = y_true != 0
    if mask.sum() == 0:
        return np.nan
    return np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100

def create_multivariate_dataset(y_scaled, exog_scaled, look_back):
    """
    y_scaled: (n,1)
    exog_scaled: (n, k)
    returns X shape (n-look_back, look_back, 1+k), y: (n-look_back,)
    """
    n = len(y_scaled)
    k = exog_scaled.shape[1] if exog_scaled is not None else 0
    X, Y = [], []
    for i in range(n - look_back):
        seq_y = y_scaled[i:i+look_back, 0]
        if k > 0:
            seq_exog = exog_scaled[i:i+look_back, :]
            seq = np.concatenate([seq_y.reshape(-1,1), seq_exog], axis=1)
        else:
            seq = seq_y.reshape(-1,1)
        X.append(seq)
        Y.append(y_scaled[i+look_back, 0])
    return np.array(X), np.array(Y)

def positional_encoding(seq_len, d_model):
    """Sinusoidal positional encoding (seq_len, d_model)"""
    pos = np.arange(seq_len)[:, np.newaxis]
    i = np.arange(d_model)[np.newaxis, :]
    angle_rates = 1 / np.power(10000, (2 * (i//2)) / np.float32(d_model))
    angle_rads = pos * angle_rates
    # apply sin to even indices in the array; cos to odd indices
    s = np.zeros_like(angle_rads)
    s[:, 0::2] = np.sin(angle_rads[:, 0::2])
    s[:, 1::2] = np.cos(angle_rads[:, 1::2])
    return tf.cast(s, dtype=tf.float32)

# -----------------------------
# Build TAT+MHA model
# -----------------------------
def build_tat_mha_model(input_shape, d_model=64, num_heads=4, ff_dim=128, dropout_rate=0.15):
    """
    input_shape: (seq_len, num_features)
    returns compiled Keras model
    """
    seq_len, num_feat = input_shape
    inp = layers.Input(shape=(seq_len, num_feat))  # (B, S, F)

    # project features to d_model
    x = layers.Dense(d_model)(inp)  # (B, S, d_model)

    # add positional encoding
    pos_enc = positional_encoding(seq_len, d_model)
    x = x + pos_enc  # broadcast ok

    # Transformer Block(s) - we use one or two blocks
    # Multi-head attention (temporal self-attention)
    attn_out = layers.MultiHeadAttention(num_heads=num_heads, key_dim=d_model)(x, x)
    attn_out = layers.Dropout(dropout_rate)(attn_out)
    x = layers.LayerNormalization(epsilon=1e-6)(x + attn_out)

    # Feed-forward
    ff = layers.Dense(ff_dim, activation='relu')(x)
    ff = layers.Dense(d_model)(ff)
    ff = layers.Dropout(dropout_rate)(ff)
    x = layers.LayerNormalization(epsilon=1e-6)(x + ff)

    # (Optional) additional MHA block
    attn_out2 = layers.MultiHeadAttention(num_heads=num_heads, key_dim=d_model)(x, x)
    attn_out2 = layers.Dropout(dropout_rate)(attn_out2)
    x = layers.LayerNormalization(epsilon=1e-6)(x + attn_out2)
    ff2 = layers.Dense(ff_dim, activation='relu')(x)
    ff2 = layers.Dense(d_model)(ff2)
    ff2 = layers.Dropout(dropout_rate)(ff2)
    x = layers.LayerNormalization(epsilon=1e-6)(x + ff2)

    # Pool and output
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dense(ff_dim//2, activation='relu')(x)
    x = layers.Dropout(dropout_rate)(x)
    out = layers.Dense(1)(x)

    model = models.Model(inputs=inp, outputs=out)
    model.compile(optimizer=optimizers.Adam(1e-3), loss='mse', metrics=['mae'])
    return model

# -----------------------------
# Main loop: process each weekly crop CSV
# -----------------------------
metrics_list = []

for file in sorted(os.listdir(input_folder)):
    if not file.endswith(".csv"):
        continue

    print(f"\n🚀 Processing Crop File: {file}")
    file_path = os.path.join(input_folder, file)

    # -------------------------
    # Load & parse
    # -------------------------
    df = pd.read_csv(file_path)
    df['Date'] = parse_dates_safe(df['Date'])
    df = df.sort_values('Date')
    # aggregate duplicate dates (take mean of numeric columns)
    df = df.groupby('Date', as_index=False).mean(numeric_only=True)
    df.set_index('Date', inplace=True)

    # set weekly frequency (align to week)
    df = df.asfreq('W')
    # ensure Average Price exists
    if 'Average Price' not in df.columns:
        print(f"⚠ 'Average Price' not found in {file}, skipping.")
        continue

    # fill missing price values
    df['Average Price'] = df['Average Price'].ffill().bfill().fillna(df['Average Price'].mean())

    # -------------------------
    # Feature engineering (weekly)
    # -------------------------
    df['Lag_1'] = df['Average Price'].shift(1)
    df['Lag_4'] = df['Average Price'].shift(4)
    df['MA_4'] = df['Average Price'].rolling(window=4).mean()
    df['MA_12'] = df['Average Price'].rolling(window=12).mean()

    # backfill/forward fill any remaining NaNs in features
    df[['Lag_1','Lag_4','MA_4','MA_12']] = df[['Lag_1','Lag_4','MA_4','MA_12']].bfill().ffill()

    # -------------------------
    # Prepare arrays & scalers
    # -------------------------
    # We'll use target scaler to inverse predictions later
    y = df[['Average Price']].values.astype('float32')
    exog_cols = ['Lag_1','Lag_4','MA_4','MA_12']
    exog = df[exog_cols].values.astype('float32')

    # scale
    y_scaler = MinMaxScaler()
    exog_scaler = MinMaxScaler()
    y_scaled = y_scaler.fit_transform(y)
    exog_scaled = exog_scaler.fit_transform(exog)

    # sequences
    X_all, y_all = create_multivariate_dataset(y_scaled, exog_scaled, look_back)
    if len(X_all) == 0:
        print(f"⚠ Not enough data for look_back={look_back} in {file}. Skipping.")
        continue

    # train/val split (chronological)
    val_size = int(len(X_all) * val_split)
    train_size = len(X_all) - val_size
    X_train = X_all[:train_size]
    y_train = y_all[:train_size]
    X_val = X_all[train_size:]
    y_val = y_all[train_size:]

    # -------------------------
    # Build model & train
    # -------------------------
    input_shape = (X_train.shape[1], X_train.shape[2])
    model = build_tat_mha_model(input_shape, d_model=d_model, num_heads=num_heads, ff_dim=ff_dim, dropout_rate=dropout_rate)

    es = callbacks.EarlyStopping(monitor='val_loss', patience=8, restore_best_weights=True)
    rl = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=4, min_lr=1e-6)

    history = model.fit(X_train, y_train,
                        validation_data=(X_val, y_val),
                        epochs=epochs,
                        batch_size=batch_size,
                        callbacks=[es, rl],
                        verbose=1)

    # -------------------------
    # Predict on all sequences and align with original index
    # -------------------------
    preds_scaled = model.predict(X_all, batch_size=batch_size)
    preds = y_scaler.inverse_transform(preds_scaled).flatten()

    # create Predicted column with NaNs for first look_back rows
    df['Predicted'] = np.nan
    start_idx = look_back
    df.iloc[start_idx : start_idx + len(preds), df.columns.get_loc('Predicted')] = preds

    # round to 2 decimals
    for col in ['Average Price','Predicted','Lag_1','Lag_4','MA_4','MA_12']:
        if col in df.columns:
            df[col] = df[col].round(2)

    # -------------------------
    # Metrics (on rows where prediction exists)
    # -------------------------
    mask = ~np.isnan(df['Predicted'].values)
    y_true = df.loc[mask, 'Average Price'].values
    y_pred = df.loc[mask, 'Predicted'].values

    mae = round(mean_absolute_error(y_true, y_pred), 2)
    rmse = round(np.sqrt(mean_squared_error(y_true, y_pred)), 2)
    r2 = round(r2_score(y_true, y_pred), 4)
    mape = round(mean_absolute_percentage_error(y_true, y_pred), 2)
    accuracy = round(100 - mape, 2)

    metrics_list.append({
        'Crop': file.replace('.csv',''),
        'MAE': mae,
        'RMSE': rmse,
        'R2': r2,
        'MAPE(%)': mape,
        'Accuracy(%)': accuracy
    })

    print(f"✅ {file} | MAE={mae}, RMSE={rmse}, R²={r2}, MAPE={mape}%, Accuracy={accuracy}%")

    # -------------------------
    # Save model, CSV, graph
    # -------------------------
    model_save_path = os.path.join(output_models, file.replace('.csv','_tat_mha_weekly.keras'))
    model.save(model_save_path)

    out_csv = os.path.join(output_csv, file.replace('.csv','_tat_mha_weekly_updated.csv'))
    df_reset = df.reset_index()
    df_reset.to_csv(out_csv, index=False, float_format='%.2f')

    # graph
    plt.figure(figsize=(12,6))
    plt.plot(df_reset['Date'], df_reset['Average Price'], label='Actual', color='blue')
    plt.plot(df_reset['Date'], df_reset['Predicted'], label='Predicted (TAT+MHA)', color='red', linestyle='dashed')
    plt.plot(df_reset['Date'], df_reset['MA_4'], label='MA_4', color='green', linestyle='--')
    plt.plot(df_reset['Date'], df_reset['MA_12'], label='MA_12', color='orange', linestyle='--')
    plt.title(f"TAT+MHA Weekly Forecast - {file}")
    plt.xlabel("Date")
    plt.ylabel("Average Price")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(os.path.join(output_graphs, file.replace('.csv','_tat_mha_weekly_graph.png')))
    plt.close()

    # cleanup
    del model, X_all, X_train, X_val, y_all, y_train, y_val, preds_scaled, preds
    gc.collect()

# -----------------------------
# Save metrics summary (all crops)
# -----------------------------
metrics_df = pd.DataFrame(metrics_list).round(2)
metrics_df.to_csv(metrics_file, index=False)
print(f"\n📊 All crops processed — metrics saved to {metrics_file}")
print(metrics_df)


🚀 Processing Crop File: capsicum_2010_2024_master_weekly.csv
Epoch 1/60
21/21 ━━━━━━━━━━━━━━━━━━━━ 33s 237ms/step - loss: 0.3285 - mae: 0.3916 - val_loss: 0.0301 - val_mae: 0.1328 - learning_rate: 0.0010
Epoch 2/60
21/21 ━━━━━━━━━━━━━━━━━━━━ 2s 66ms/step - loss: 0.0315 - mae: 0.1415 - val_loss: 0.0235 - val_mae: 0.1148 - learning_rate: 0.0010
Epoch 3/60
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 50ms/step - loss: 0.0217 - mae: 0.1181 - val_loss: 0.0203 - val_mae: 0.1066 - learning_rate: 0.0010
Epoch 4/60
21/21 ━━━━━━━━━━━━━━━━━━━━ 2s 93ms/step - loss: 0.0205 - mae: 0.1132 - val_loss: 0.0259 - val_mae: 0.1218 - learning_rate: 0.0010
Epoch 5/60
21/21 ━━━━━━━━━━━━━━━━━━━━ 2s 85ms/step - loss: 0.0221 - mae: 0.1170 - val_loss: 0.0191 - val_mae: 0.1035 - learning_rate: 0.0010
Epoch 6/60
21/21 ━━━━━━━━━━━━━━━━━━━━ 2s 78ms/step - loss: 0.0205 - mae: 0.1146 - val_loss: 0.0230 - val_mae: 0.1149 - learning_rate: 0.0010
Epoch 7/60
21/21 ━━━━━━━━━━━━━━━━━━━━ 2s 71ms/step - loss: 0.0184 - mae: 0.1063 - val_loss

In [ ]:
#TAT+MQA

In [6]:
import os, gc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers, callbacks

# -----------------------------
# Paths / outputs
# -----------------------------
input_folder = "dataset"
output_models = "tat_mqa_output_models_weekly"
output_csv = "tat_mqa_output_csv_weekly"
output_graphs = "tat_mqa_output_graphs_weekly"
metrics_file = "tat_mqa_metrics_weekly.csv"

os.makedirs(output_models, exist_ok=True)
os.makedirs(output_csv, exist_ok=True)
os.makedirs(output_graphs, exist_ok=True)

# -----------------------------
# Config (tweak if needed)
# -----------------------------
look_back = 30       # weeks of history used as input
batch_size = 32
epochs = 60
val_split = 0.15
d_model = 64
num_heads = 4        # used for key/value dim split if needed, MQA uses multiple Q projections
num_q_heads = 4      # how many different Q projections (multi-query)
ff_dim = 128
dropout_rate = 0.15
seed = 42

tf.random.set_seed(seed)
np.random.seed(seed)

# -----------------------------
# Helpers
# -----------------------------
def parse_dates_safe(s):
    try:
        return pd.to_datetime(s, dayfirst=True)
    except:
        return pd.to_datetime(s, dayfirst=False)

def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    mask = y_true != 0
    if mask.sum() == 0:
        return np.nan
    return np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100

def create_multivariate_dataset(y_scaled, exog_scaled, look_back):
    n = len(y_scaled)
    k = exog_scaled.shape[1] if exog_scaled is not None else 0
    X, Y = [], []
    for i in range(n - look_back):
        seq_y = y_scaled[i:i+look_back, 0]
        if k > 0:
            seq_exog = exog_scaled[i:i+look_back, :]
            seq = np.concatenate([seq_y.reshape(-1,1), seq_exog], axis=1)
        else:
            seq = seq_y.reshape(-1,1)
        X.append(seq)
        Y.append(y_scaled[i+look_back, 0])
    return np.array(X), np.array(Y)

def positional_encoding(seq_len, d_model):
    pos = np.arange(seq_len)[:, np.newaxis]
    i = np.arange(d_model)[np.newaxis, :]
    angle_rates = 1 / np.power(10000, (2 * (i//2)) / np.float32(d_model))
    angle_rads = pos * angle_rates
    pe = np.zeros_like(angle_rads)
    pe[:, 0::2] = np.sin(angle_rads[:, 0::2])
    pe[:, 1::2] = np.cos(angle_rads[:, 1::2])
    return tf.cast(pe, tf.float32)

# -----------------------------
# Multi-Query Attention (MQA) Layer
# -----------------------------
class MultiQueryAttention(layers.Layer):
    def __init__(self, num_q_heads=4, key_dim=64, dropout_rate=0.1, **kwargs):
        super().__init__(**kwargs)
        self.num_q_heads = num_q_heads
        self.key_dim = key_dim
        self.dropout_rate = dropout_rate
        # we'll create Q dense layers list, and shared K,V
        self.q_layers = [layers.Dense(key_dim) for _ in range(num_q_heads)]
        self.k_dense = layers.Dense(key_dim)
        self.v_dense = layers.Dense(key_dim)
        self.out_dense = layers.Dense(key_dim)
        self.dropout = layers.Dropout(dropout_rate)

    def call(self, x, training=False):
        # x: (batch, seq_len, d_model)
        K = self.k_dense(x)  # (B, S, key_dim)
        V = self.v_dense(x)  # (B, S, key_dim)
        head_outputs = []
        for q_layer in self.q_layers:
            Q = q_layer(x)  # (B, S, key_dim)
            # scaled dot-product: Q @ K^T
            scores = tf.matmul(Q, K, transpose_b=True) / tf.math.sqrt(tf.cast(self.key_dim, tf.float32))
            attn = tf.nn.softmax(scores, axis=-1)  # (B, S, S)
            out = tf.matmul(attn, V)  # (B, S, key_dim)
            head_outputs.append(out)
        concat = tf.concat(head_outputs, axis=-1)  # (B, S, key_dim * num_q_heads)
        proj = self.out_dense(concat)               # project back to key_dim (or d_model)
        proj = self.dropout(proj, training=training)
        return proj  # (B, S, key_dim)

    def get_config(self):
        config = super().get_config()
        config.update({
            "num_q_heads": self.num_q_heads,
            "key_dim": self.key_dim,
            "dropout_rate": self.dropout_rate
        })
        return config

# -----------------------------
# Build TAT + MQA model
# -----------------------------
def build_tat_mqa_model(input_shape, d_model=64, num_q_heads=4, ff_dim=128, dropout_rate=0.15):
    seq_len, num_feat = input_shape
    inp = layers.Input(shape=(seq_len, num_feat))  # (B,S,F)
    # project input features to d_model for residual compatibility
    x = layers.Dense(d_model)(inp)                # (B,S,d_model)
    # add positional enc
    pe = positional_encoding(seq_len, d_model)
    x = x + pe

    # Multi-Query Attention block
    mqa = MultiQueryAttention(num_q_heads=num_q_heads, key_dim=d_model, dropout_rate=dropout_rate)
    attn_out = mqa(x)                              # (B,S,d_model)
    x = layers.LayerNormalization(epsilon=1e-6)(x + attn_out)

    # Feed-forward
    ff = layers.Dense(ff_dim, activation='relu')(x)
    ff = layers.Dense(d_model)(ff)
    ff = layers.Dropout(dropout_rate)(ff)
    x = layers.LayerNormalization(epsilon=1e-6)(x + ff)

    # Optional second MHA block (temporal)
    attn2 = layers.MultiHeadAttention(num_heads=num_heads, key_dim=d_model)(x, x)
    attn2 = layers.Dropout(dropout_rate)(attn2)
    x = layers.LayerNormalization(epsilon=1e-6)(x + attn2)
    ff2 = layers.Dense(ff_dim, activation='relu')(x)
    ff2 = layers.Dense(d_model)(ff2)
    ff2 = layers.Dropout(dropout_rate)(ff2)
    x = layers.LayerNormalization(epsilon=1e-6)(x + ff2)

    # Pool and output
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dense(ff_dim//2, activation='relu')(x)
    x = layers.Dropout(dropout_rate)(x)
    out = layers.Dense(1)(x)

    model = models.Model(inputs=inp, outputs=out)
    model.compile(optimizer=optimizers.Adam(1e-3), loss='mse', metrics=['mae'])
    return model

# -----------------------------
# Main loop — process weekly crop files
# -----------------------------
metrics_list = []

for file in sorted(os.listdir(input_folder)):
    if not file.endswith(".csv"):
        continue
    print(f"\n🚀 Processing Crop File: {file}")
    file_path = os.path.join(input_folder, file)
    df = pd.read_csv(file_path)

    # parse dates and sort
    df['Date'] = parse_dates_safe(df['Date'])
    df = df.sort_values('Date')
    df = df.groupby('Date', as_index=False).mean(numeric_only=True)
    df.set_index('Date', inplace=True)

    # weekly frequency
    df = df.asfreq('W')

    # ensure Average Price exists and fill
    if 'Average Price' not in df.columns:
        print(f"⚠ 'Average Price' not found in {file}. Skipping.")
        continue
    df['Average Price'] = df['Average Price'].ffill().bfill().fillna(df['Average Price'].mean())

    # feature engineering (weekly)
    df['Lag_1'] = df['Average Price'].shift(1)
    df['Lag_4'] = df['Average Price'].shift(4)
    df['MA_4'] = df['Average Price'].rolling(window=4).mean()
    df['MA_12'] = df['Average Price'].rolling(window=12).mean()
    df[['Lag_1','Lag_4','MA_4','MA_12']] = df[['Lag_1','Lag_4','MA_4','MA_12']].bfill().ffill()

    # prepare arrays & scalers
    y = df[['Average Price']].values.astype('float32')
    exog_cols = ['Lag_1','Lag_4','MA_4','MA_12']
    exog = df[exog_cols].values.astype('float32')

    y_scaler = MinMaxScaler()
    exog_scaler = MinMaxScaler()
    y_scaled = y_scaler.fit_transform(y)
    exog_scaled = exog_scaler.fit_transform(exog)

    # create sequences
    X_all, y_all = create_multivariate_dataset(y_scaled, exog_scaled, look_back)
    if len(X_all) == 0:
        print(f"⚠ Not enough data after look_back={look_back} in {file}. Skipping.")
        continue

    # train/val split
    val_size = int(len(X_all) * val_split)
    train_size = len(X_all) - val_size
    X_train, y_train = X_all[:train_size], y_all[:train_size]
    X_val, y_val = X_all[train_size:], y_all[train_size:]

    # build & train model
    input_shape = (X_train.shape[1], X_train.shape[2])
    model = build_tat_mqa_model(input_shape, d_model=d_model, num_q_heads=num_q_heads, ff_dim=ff_dim, dropout_rate=dropout_rate)

    es = callbacks.EarlyStopping(monitor='val_loss', patience=8, restore_best_weights=True)
    rl = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=4, min_lr=1e-6)

    history = model.fit(X_train, y_train,
                        validation_data=(X_val, y_val),
                        epochs=epochs,
                        batch_size=batch_size,
                        callbacks=[es, rl],
                        verbose=1)

    # predict on all sequences and align
    preds_scaled = model.predict(X_all, batch_size=batch_size)
    preds = y_scaler.inverse_transform(preds_scaled).flatten()

    df['Predicted'] = np.nan
    start_idx = look_back
    df.iloc[start_idx:start_idx + len(preds), df.columns.get_loc('Predicted')] = preds

    # round numeric columns to 2 decimals
    for col in ['Average Price','Predicted','Lag_1','Lag_4','MA_4','MA_12']:
        if col in df.columns:
            df[col] = df[col].round(2)

    # compute metrics on rows with predictions
    mask = ~np.isnan(df['Predicted'].values)
    y_true = df.loc[mask, 'Average Price'].values
    y_pred = df.loc[mask, 'Predicted'].values

    mae = round(mean_absolute_error(y_true, y_pred), 2)
    rmse = round(np.sqrt(mean_squared_error(y_true, y_pred)), 2)
    r2 = round(r2_score(y_true, y_pred), 4)
    mape = round(mean_absolute_percentage_error(y_true, y_pred), 2)
    accuracy = round(100 - mape, 2)

    metrics_list.append({
        'Crop': file.replace('.csv',''),
        'MAE': mae,
        'RMSE': rmse,
        'R2': r2,
        'MAPE(%)': mape,
        'Accuracy(%)': accuracy
    })

    print(f"✅ {file} | MAE={mae}, RMSE={rmse}, R²={r2}, MAPE={mape}%, Accuracy={accuracy}%")

    # save model, CSV, graph
    model_save_path = os.path.join(output_models, file.replace('.csv','_tat_mqa_weekly.keras'))
    model.save(model_save_path)

    df_reset = df.reset_index()
    df_reset.to_csv(os.path.join(output_csv, file.replace('.csv','_tat_mqa_weekly_updated.csv')), index=False, float_format='%.2f')

    plt.figure(figsize=(12,6))
    plt.plot(df_reset['Date'], df_reset['Average Price'], label='Actual', color='blue')
    plt.plot(df_reset['Date'], df_reset['Predicted'], label='Predicted (TAT+MQA)', color='red', linestyle='dashed')
    plt.plot(df_reset['Date'], df_reset['MA_4'], label='MA_4', color='green', linestyle='--')
    plt.plot(df_reset['Date'], df_reset['MA_12'], label='MA_12', color='orange', linestyle='--')
    plt.title(f"TAT+MQA Weekly Forecast - {file}")
    plt.xlabel("Date")
    plt.ylabel("Average Price")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(os.path.join(output_graphs, file.replace('.csv','_tat_mqa_weekly_graph.png')))
    plt.close()

    # cleanup
    del model, X_all, X_train, X_val, y_all, y_train, y_val, preds_scaled, preds
    gc.collect()

# save consolidated metrics
metrics_df = pd.DataFrame(metrics_list).round(2)
metrics_df.to_csv(metrics_file, index=False)
print(f"\n📊 All crops processed — metrics saved to {metrics_file}")
print(metrics_df)


🚀 Processing Crop File: capsicum_2010_2024_master_weekly.csv

Epoch 1/60
21/21 ━━━━━━━━━━━━━━━━━━━━ 27s 124ms/step - loss: 0.5550 - mae: 0.5596 - val_loss: 0.2944 - val_mae: 0.5278 - learning_rate: 0.0010
Epoch 2/60
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 50ms/step - loss: 0.0782 - mae: 0.2173 - val_loss: 0.0169 - val_mae: 0.0997 - learning_rate: 0.0010
Epoch 3/60
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 59ms/step - loss: 0.0387 - mae: 0.1548 - val_loss: 0.0218 - val_mae: 0.1101 - learning_rate: 0.0010
Epoch 4/60
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 57ms/step - loss: 0.0264 - mae: 0.1258 - val_loss: 0.0190 - val_mae: 0.1036 - learning_rate: 0.0010
Epoch 5/60
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 53ms/step - loss: 0.0235 - mae: 0.1220 - val_loss: 0.0349 - val_mae: 0.1493 - learning_rate: 0.0010
Epoch 6/60
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 39ms/step - loss: 0.0215 - mae: 0.1165 - val_loss: 0.0266 - val_mae: 0.1265 - learning_rate: 0.0010
Epoch 7/60
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 51ms/step - loss: 0.0191 - mae: 0.1088 - val_los

In [ ]:
#TAT+GQA

In [7]:
import os
import gc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers, callbacks

# -----------------------------
# Paths / outputs
# -----------------------------
input_folder = "dataset"   # folder with weekly CSVs
output_models = "tat_gqa_output_models_weekly"
output_csv = "tat_gqa_output_csv_weekly"
output_graphs = "tat_gqa_output_graphs_weekly"
metrics_file = "tat_gqa_metrics_weekly.csv"

os.makedirs(output_models, exist_ok=True)
os.makedirs(output_csv, exist_ok=True)
os.makedirs(output_graphs, exist_ok=True)

# -----------------------------
# Config (tweak if needed)
# -----------------------------
look_back = 30       # weeks of history used as input
batch_size = 32
epochs = 60
val_split = 0.15
d_model = 64
num_groups = 2       # number of query groups in GQA
ff_dim = 128
dropout_rate = 0.15
seed = 42

tf.random.set_seed(seed)
np.random.seed(seed)

# -----------------------------
# Helpers
# -----------------------------
def parse_dates_safe(s):
    try:
        return pd.to_datetime(s, dayfirst=True)
    except:
        return pd.to_datetime(s, dayfirst=False)

def mean_absolute_percentage_error(y_true, y_pred):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    mask = y_true != 0
    if mask.sum() == 0:
        return np.nan
    return np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100

def create_multivariate_dataset(y_scaled, exog_scaled, look_back):
    n = len(y_scaled)
    k = exog_scaled.shape[1] if exog_scaled is not None else 0
    X, Y = [], []
    for i in range(n - look_back):
        seq_y = y_scaled[i:i+look_back, 0]
        if k > 0:
            seq_exog = exog_scaled[i:i+look_back, :]
            seq = np.concatenate([seq_y.reshape(-1,1), seq_exog], axis=1)
        else:
            seq = seq_y.reshape(-1,1)
        X.append(seq)
        Y.append(y_scaled[i+look_back, 0])
    return np.array(X), np.array(Y)

def positional_encoding(seq_len, d_model):
    pos = np.arange(seq_len)[:, np.newaxis]
    i = np.arange(d_model)[np.newaxis, :]
    angle_rates = 1 / np.power(10000, (2 * (i//2)) / np.float32(d_model))
    angle_rads = pos * angle_rates
    pe = np.zeros_like(angle_rads)
    pe[:, 0::2] = np.sin(angle_rads[:, 0::2])
    pe[:, 1::2] = np.cos(angle_rads[:, 1::2])
    return tf.cast(pe, tf.float32)

# -----------------------------
# Grouped Query Attention Layer (safe for graph mode)
# -----------------------------
@tf.keras.utils.register_keras_serializable()
class GroupedQueryAttention(layers.Layer):
    def __init__(self, num_groups=2, key_dim=64, dropout_rate=0.1, **kwargs):
        super().__init__(**kwargs)
        self.num_groups = int(num_groups)
        self.key_dim = int(key_dim)
        self.dropout_rate = dropout_rate
        # Create layer objects (they will be built automatically)
        self.k_dense = layers.Dense(self.key_dim, name="gqa_k_dense")
        self.v_dense = layers.Dense(self.key_dim, name="gqa_v_dense")
        self.q_dense_groups = [layers.Dense(self.key_dim, name=f"gqa_q_dense_{i}") for i in range(self.num_groups)]
        self.output_dense = layers.Dense(self.key_dim, name="gqa_out_dense")
        self.dropout = layers.Dropout(dropout_rate)

    def build(self, input_shape):
        # nothing else to build; children layers will handle their weights
        super().build(input_shape)

    def call(self, x, training=False):
        # x: (batch, seq, d_model)
        K = self.k_dense(x)              # (B, S, key_dim)
        V = self.v_dense(x)              # (B, S, key_dim)
        head_outputs = []
        sqrt_k = tf.math.sqrt(tf.cast(self.key_dim, tf.float32))
        for q_layer in self.q_dense_groups:
            Q = q_layer(x)               # (B, S, key_dim)
            # scores: (B, S, S)
            scores = tf.matmul(Q, K, transpose_b=True) / sqrt_k
            attn_weights = tf.nn.softmax(scores, axis=-1)
            attn_out = tf.matmul(attn_weights, V)  # (B, S, key_dim)
            head_outputs.append(attn_out)
        # concat along last axis (B, S, key_dim * num_groups)
        concat = tf.concat(head_outputs, axis=-1)
        proj = self.output_dense(concat)   # project back to key_dim (or d_model)
        proj = self.dropout(proj, training=training)
        return proj  # (B, S, key_dim)

    def get_config(self):
        cfg = super().get_config()
        cfg.update({"num_groups": self.num_groups, "key_dim": self.key_dim, "dropout_rate": self.dropout_rate})
        return cfg

# -----------------------------
# Build TAT + GQA model
# -----------------------------
def build_tat_gqa_model(input_shape, d_model=64, num_groups=2, ff_dim=128, dropout_rate=0.15):
    seq_len, num_feat = input_shape
    inp = layers.Input(shape=(seq_len, num_feat), name="input_seq")   # (B, S, F)
    # project features -> d_model
    x = layers.Dense(d_model, name="proj_in")(inp)                   # (B, S, d_model)
    # add positional encoding
    pos_enc = positional_encoding(seq_len, d_model)
    x = x + pos_enc

    # Grouped Query Attention block
    gqa = GroupedQueryAttention(num_groups=num_groups, key_dim=d_model, dropout_rate=dropout_rate)
    attn_out = gqa(x)                  # (B, S, d_model)
    x = layers.LayerNormalization(epsilon=1e-6, name="ln_gqa")(x + attn_out)

    # Feed-forward
    ff = layers.Dense(ff_dim, activation='relu', name="ff1")(x)
    ff = layers.Dense(d_model, name="ff2")(ff)
    ff = layers.Dropout(dropout_rate)(ff)
    x = layers.LayerNormalization(epsilon=1e-6, name="ln_ff")(x + ff)

    # Add a temporal MultiHeadAttention block for extra temporal mixing
    attn2 = layers.MultiHeadAttention(num_heads=4, key_dim=d_model)(x, x)
    attn2 = layers.Dropout(dropout_rate)(attn2)
    x = layers.LayerNormalization(epsilon=1e-6, name="ln_mha")(x + attn2)
    ff2 = layers.Dense(ff_dim, activation='relu')(x)
    ff2 = layers.Dense(d_model)(ff2)
    ff2 = layers.Dropout(dropout_rate)(ff2)
    x = layers.LayerNormalization(epsilon=1e-6)(x + ff2)

    # Pool & output
    x = layers.GlobalAveragePooling1D(name="global_pool")(x)
    x = layers.Dense(ff_dim//2, activation='relu')(x)
    x = layers.Dropout(dropout_rate)(x)
    out = layers.Dense(1, name="out")(x)

    model = models.Model(inputs=inp, outputs=out, name="TAT_GQA_weekly")
    model.compile(optimizer=optimizers.Adam(1e-3), loss='mse', metrics=['mae'])
    return model

# -----------------------------
# Main loop
# -----------------------------
metrics_list = []

for file in sorted(os.listdir(input_folder)):
    if not file.endswith(".csv"):
        continue
    print(f"\n🚀 Processing Crop File: {file}")
    file_path = os.path.join(input_folder, file)

    # load
    df = pd.read_csv(file_path)
    df['Date'] = parse_dates_safe(df['Date'])
    df = df.sort_values('Date')
    df = df.groupby('Date', as_index=False).mean(numeric_only=True)
    df.set_index('Date', inplace=True)

    # weekly frequency alignment
    df = df.asfreq('W')
    if 'Average Price' not in df.columns:
        print(f"⚠ 'Average Price' missing in {file}, skipping.")
        continue
    df['Average Price'] = df['Average Price'].ffill().bfill().fillna(df['Average Price'].mean())

    # features: weekly equivalents
    df['Lag_1'] = df['Average Price'].shift(1)
    df['Lag_4'] = df['Average Price'].shift(4)
    df['MA_4']  = df['Average Price'].rolling(window=4).mean()
    df['MA_12'] = df['Average Price'].rolling(window=12).mean()
    # fill feature NaNs
    df[['Lag_1','Lag_4','MA_4','MA_12']] = df[['Lag_1','Lag_4','MA_4','MA_12']].bfill().ffill()

    # prepare arrays and scalers
    y = df[['Average Price']].values.astype('float32')
    exog_cols = ['Lag_1','Lag_4','MA_4','MA_12']
    exog = df[exog_cols].values.astype('float32')

    y_scaler = MinMaxScaler()
    exog_scaler = MinMaxScaler()
    y_scaled = y_scaler.fit_transform(y)
    exog_scaled = exog_scaler.fit_transform(exog)

    # sequences (multivariate)
    X_all, y_all = create_multivariate_dataset(y_scaled, exog_scaled, look_back)
    if len(X_all) == 0:
        print(f"⚠ Not enough data after look_back={look_back} in {file}. Skipping.")
        continue

    # train/val split
    val_size = int(len(X_all) * val_split)
    train_size = len(X_all) - val_size
    X_train, y_train = X_all[:train_size], y_all[:train_size]
    X_val, y_val     = X_all[train_size:], y_all[train_size:]

    # build model
    input_shape = (X_train.shape[1], X_train.shape[2])
    model = build_tat_gqa_model(input_shape, d_model=d_model, num_groups=num_groups, ff_dim=ff_dim, dropout_rate=dropout_rate)

    # callbacks
    es = callbacks.EarlyStopping(monitor='val_loss', patience=8, restore_best_weights=True)
    rl = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=4, min_lr=1e-6)

    # train
    history = model.fit(X_train, y_train,
                        validation_data=(X_val, y_val),
                        epochs=epochs, batch_size=batch_size,
                        callbacks=[es, rl], verbose=1)

    # predict on all sequences and align
    preds_scaled = model.predict(X_all, batch_size=batch_size)
    preds = y_scaler.inverse_transform(preds_scaled).flatten()

    df['Predicted'] = np.nan
    start_idx = look_back
    df.iloc[start_idx:start_idx + len(preds), df.columns.get_loc('Predicted')] = preds

    # round to 2 decimals
    for col in ['Average Price','Predicted','Lag_1','Lag_4','MA_4','MA_12']:
        if col in df.columns:
            df[col] = df[col].round(2)

    # metrics where prediction exists
    mask = ~np.isnan(df['Predicted'].values)
    y_true = df.loc[mask, 'Average Price'].values
    y_pred = df.loc[mask, 'Predicted'].values

    mae = round(mean_absolute_error(y_true, y_pred), 2)
    rmse = round(np.sqrt(mean_squared_error(y_true, y_pred)), 2)
    r2 = round(r2_score(y_true, y_pred), 4)
    mape = round(mean_absolute_percentage_error(y_true, y_pred), 2)
    accuracy = round(100 - mape, 2)

    metrics_list.append({
        'Crop': file.replace('.csv',''),
        'MAE': mae,
        'RMSE': rmse,
        'R2': r2,
        'MAPE(%)': mape,
        'Accuracy(%)': accuracy
    })

    print(f"✅ {file} | MAE={mae}, RMSE={rmse}, R²={r2}, MAPE={mape}%, Accuracy={accuracy}%")

    # save model, csv, graph
    model_save_path = os.path.join(output_models, file.replace('.csv','_tat_gqa_weekly.keras'))
    model.save(model_save_path)

    out_csv = os.path.join(output_csv, file.replace('.csv','_tat_gqa_weekly_updated.csv'))
    df_reset = df.reset_index()
    df_reset.to_csv(out_csv, index=False, float_format='%.2f')

    plt.figure(figsize=(12,6))
    plt.plot(df_reset['Date'], df_reset['Average Price'], label='Actual', color='blue')
    plt.plot(df_reset['Date'], df_reset['Predicted'], label='Predicted (TAT+GQA)', color='red', linestyle='dashed')
    plt.plot(df_reset['Date'], df_reset['MA_4'], label='MA_4', color='green', linestyle='--')
    plt.plot(df_reset['Date'], df_reset['MA_12'], label='MA_12', color='orange', linestyle='--')
    plt.title(f"TAT+GQA Weekly Forecast - {file}")
    plt.xlabel("Date")
    plt.ylabel("Average Price")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(os.path.join(output_graphs, file.replace('.csv','_tat_gqa_weekly_graph.png')))
    plt.close()

    # cleanup
    del model, X_all, X_train, X_val, y_all, y_train, y_val, preds_scaled, preds
    gc.collect()

# save consolidated metrics
metrics_df = pd.DataFrame(metrics_list).round(2)
metrics_df.to_csv(metrics_file, index=False)
print(f"\n📊 All crops processed — metrics saved to {metrics_file}")
print(metrics_df)


🚀 Processing Crop File: capsicum_2010_2024_master_weekly.csv
Epoch 1/60
21/21 ━━━━━━━━━━━━━━━━━━━━ 28s 125ms/step - loss: 0.1826 - mae: 0.3188 - val_loss: 0.0651 - val_mae: 0.2220 - learning_rate: 0.0010
Epoch 2/60
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 35ms/step - loss: 0.0387 - mae: 0.1596 - val_loss: 0.0909 - val_mae: 0.2738 - learning_rate: 0.0010
Epoch 3/60
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 39ms/step - loss: 0.0260 - mae: 0.1288 - val_loss: 0.0559 - val_mae: 0.1998 - learning_rate: 0.0010
Epoch 4/60
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 38ms/step - loss: 0.0222 - mae: 0.1205 - val_loss: 0.0310 - val_mae: 0.1369 - learning_rate: 0.0010
Epoch 5/60
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step - loss: 0.0194 - mae: 0.1098 - val_loss: 0.0165 - val_mae: 0.1011 - learning_rate: 0.0010
Epoch 6/60
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 38ms/step - loss: 0.0201 - mae: 0.1132 - val_loss: 0.0160 - val_mae: 0.1020 - learning_rate: 0.0010
Epoch 7/60
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 38ms/step - loss: 0.0213 - mae: 0.1154 - val_loss

In [ ]:
#TAT+HA

In [8]:
import os
import gc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers, callbacks

# -----------------------------
# Paths / outputs
# -----------------------------
input_folder = "dataset"   # folder with weekly CSVs
output_models = "tat_ha_output_models_weekly"
output_csv = "tat_ha_output_csv_weekly"
output_graphs = "tat_ha_output_graphs_weekly"
metrics_file = "tat_ha_metrics_weekly.csv"

os.makedirs(output_models, exist_ok=True)
os.makedirs(output_csv, exist_ok=True)
os.makedirs(output_graphs, exist_ok=True)

# -----------------------------
# Config (tweak if needed)
# -----------------------------
look_back = 30       # weeks of history used as input
batch_size = 32
epochs = 60
val_split = 0.15
d_model = 64
num_heads = 4
ff_dim = 128
dropout_rate = 0.15
seed = 42

tf.random.set_seed(seed)
np.random.seed(seed)

# -----------------------------
# Helpers
# -----------------------------
def parse_dates_safe(s):
    try:
        return pd.to_datetime(s, dayfirst=True)
    except:
        return pd.to_datetime(s, dayfirst=False)

def mean_absolute_percentage_error(y_true, y_pred):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    mask = y_true != 0
    if mask.sum() == 0:
        return np.nan
    return np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100

def create_multivariate_dataset(y_scaled, exog_scaled, look_back):
    n = len(y_scaled)
    k = exog_scaled.shape[1] if exog_scaled is not None else 0
    X, Y = [], []
    for i in range(n - look_back):
        seq_y = y_scaled[i:i+look_back, 0]
        if k > 0:
            seq_exog = exog_scaled[i:i+look_back, :]
            seq = np.concatenate([seq_y.reshape(-1,1), seq_exog], axis=1)
        else:
            seq = seq_y.reshape(-1,1)
        X.append(seq)
        Y.append(y_scaled[i+look_back, 0])
    return np.array(X), np.array(Y)

def positional_encoding(seq_len, d_model):
    pos = np.arange(seq_len)[:, np.newaxis]
    i = np.arange(d_model)[np.newaxis, :]
    angle_rates = 1 / np.power(10000, (2 * (i//2)) / np.float32(d_model))
    angle_rads = pos * angle_rates
    pe = np.zeros_like(angle_rads)
    pe[:, 0::2] = np.sin(angle_rads[:, 0::2])
    pe[:, 1::2] = np.cos(angle_rads[:, 1::2])
    return tf.cast(pe, tf.float32)

# -----------------------------
# Hierarchical Attention Layer (Graph-safe)
# -----------------------------
@tf.keras.utils.register_keras_serializable()
class HierarchicalAttention(layers.Layer):
    def __init__(self, d_model=64, num_heads=4, ff_dim=128, dropout_rate=0.15, **kwargs):
        super().__init__(**kwargs)
        self.d_model = int(d_model)
        self.num_heads = int(num_heads)
        self.ff_dim = int(ff_dim)
        self.dropout_rate = dropout_rate

        # Feature scoring: project last axis to same size (num_features) then softmax across features
        self.feature_score_dense = None  # built lazily
        # Project aggregated feature -> d_model
        self.feature_proj = layers.Dense(self.d_model)
        # Temporal MHA
        self.temporal_mha = layers.MultiHeadAttention(num_heads=self.num_heads, key_dim=self.d_model)
        # FFN
        self.ffn1 = layers.Dense(self.ff_dim, activation='relu')
        self.ffn2 = layers.Dense(self.d_model)
        # Norms & dropout
        self.ln1 = layers.LayerNormalization(epsilon=1e-6)
        self.ln2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout = layers.Dropout(self.dropout_rate)

    def build(self, input_shape):
        # input_shape = (batch, seq_len, num_features)
        num_features = int(input_shape[-1])
        # feature_score_dense projects each feature position to a scalar per feature (we produce num_features outputs)
        self.feature_score_dense = layers.Dense(num_features)
        super().build(input_shape)

    def call(self, x, training=False):
        # x: (batch, seq_len, num_features)
        # 1) Feature-level scoring per timestep
        # feature_scores: (batch, seq_len, num_features)
        feature_scores = self.feature_score_dense(x)
        # feature_weights across features
        feature_weights = tf.nn.softmax(feature_scores, axis=-1)  # softmax over features
        # weighted features
        weighted = x * feature_weights  # broadcast (batch, seq_len, num_features)
        # aggregate features into a single feature per timestep
        feature_agg = tf.reduce_sum(weighted, axis=-1, keepdims=True)  # (batch, seq_len, 1)
        # project to d_model for temporal attention
        seq_repr = self.feature_proj(feature_agg)  # (batch, seq_len, d_model)

        # 2) Temporal multi-head self-attention
        attn_out = self.temporal_mha(seq_repr, seq_repr)  # (batch, seq_len, d_model)
        attn_out = self.dropout(attn_out, training=training)
        out1 = self.ln1(seq_repr + attn_out)

        # 3) Feed-forward
        ffn = self.ffn1(out1)
        ffn = self.ffn2(ffn)
        ffn = self.dropout(ffn, training=training)
        out2 = self.ln2(out1 + ffn)

        return out2  # (batch, seq_len, d_model)

    def get_config(self):
        cfg = super().get_config()
        cfg.update({"d_model": self.d_model, "num_heads": self.num_heads, "ff_dim": self.ff_dim, "dropout_rate": self.dropout_rate})
        return cfg

# -----------------------------
# Build TAT+HA model
# -----------------------------
def build_tat_ha_model(input_shape, d_model=64, num_heads=4, ff_dim=128, dropout_rate=0.15):
    seq_len, num_feat = input_shape
    inp = layers.Input(shape=(seq_len, num_feat))
    # project input features to d_model
    x = layers.Dense(d_model)(inp)  # (B,S,d_model)
    # add positional encoding
    pe = positional_encoding(seq_len, d_model)
    x = x + pe
    # hierarchical attention block
    ha_out = HierarchicalAttention(d_model=d_model, num_heads=num_heads, ff_dim=ff_dim, dropout_rate=dropout_rate)(inp)  # note: pass original inp for feature-level scoring
    # align residuals: project inp to d_model
    inp_proj = layers.Dense(d_model)(inp)
    x = layers.LayerNormalization(epsilon=1e-6)(ha_out + inp_proj)
    # optional additional transformer layers (temporal mixing)
    attn2 = layers.MultiHeadAttention(num_heads=num_heads, key_dim=d_model)(x, x)
    attn2 = layers.Dropout(dropout_rate)(attn2)
    x = layers.LayerNormalization(epsilon=1e-6)(x + attn2)
    ff = layers.Dense(ff_dim, activation='relu')(x)
    ff = layers.Dense(d_model)(ff)
    ff = layers.Dropout(dropout_rate)(ff)
    x = layers.LayerNormalization(epsilon=1e-6)(x + ff)
    # pooling + output
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dense(ff_dim//2, activation='relu')(x)
    x = layers.Dropout(dropout_rate)(x)
    out = layers.Dense(1)(x)

    model = models.Model(inputs=inp, outputs=out)
    model.compile(optimizer=optimizers.Adam(1e-3), loss='mse', metrics=['mae'])
    return model

# -----------------------------
# Main loop: process weekly crop files
# -----------------------------
metrics_list = []

for file in sorted(os.listdir(input_folder)):
    if not file.endswith(".csv"):
        continue

    print(f"\n🚀 Processing Crop File: {file}")
    file_path = os.path.join(input_folder, file)

    # -------------------------
    # Load & parse
    # -------------------------
    df = pd.read_csv(file_path)
    df['Date'] = parse_dates_safe(df['Date'])
    df = df.sort_values('Date')
    # aggregate duplicates (mean)
    df = df.groupby('Date', as_index=False).mean(numeric_only=True)
    df.set_index('Date', inplace=True)

    # weekly frequency
    df = df.asfreq('W')

    # check column
    if 'Average Price' not in df.columns:
        print(f"⚠ 'Average Price' not found in {file}, skipping.")
        continue

    # fill missing price values
    df['Average Price'] = df['Average Price'].ffill().bfill().fillna(df['Average Price'].mean())

    # -------------------------
    # Feature engineering (weekly)
    # -------------------------
    df['Lag_1'] = df['Average Price'].shift(1)
    df['Lag_4'] = df['Average Price'].shift(4)
    df['MA_4'] = df['Average Price'].rolling(window=4).mean()
    df['MA_12'] = df['Average Price'].rolling(window=12).mean()

    # fill features
    df[['Lag_1','Lag_4','MA_4','MA_12']] = df[['Lag_1','Lag_4','MA_4','MA_12']].bfill().ffill()

    # -------------------------
    # Prepare arrays & scalers
    # -------------------------
    y = df[['Average Price']].values.astype('float32')
    exog_cols = ['Lag_1','Lag_4','MA_4','MA_12']
    exog = df[exog_cols].values.astype('float32')

    # scale
    y_scaler = MinMaxScaler()
    exog_scaler = MinMaxScaler()
    y_scaled = y_scaler.fit_transform(y)
    exog_scaled = exog_scaler.fit_transform(exog)

    # create sequences
    X_all, y_all = create_multivariate_dataset(y_scaled, exog_scaled, look_back)
    if len(X_all) == 0:
        print(f"⚠ Not enough data for look_back={look_back} in {file}. Skipping.")
        continue

    # train/val split (chronological)
    val_size = int(len(X_all) * val_split)
    train_size = len(X_all) - val_size
    X_train, y_train = X_all[:train_size], y_all[:train_size]
    X_val, y_val     = X_all[train_size:], y_all[train_size:]

    # -------------------------
    # Build model & train
    # -------------------------
    input_shape = (X_train.shape[1], X_train.shape[2])
    model = build_tat_ha_model(input_shape, d_model=d_model, num_heads=num_heads, ff_dim=ff_dim, dropout_rate=dropout_rate)

    es = callbacks.EarlyStopping(monitor='val_loss', patience=8, restore_best_weights=True)
    rl = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=4, min_lr=1e-6)

    history = model.fit(X_train, y_train,
                        validation_data=(X_val, y_val),
                        epochs=epochs,
                        batch_size=batch_size,
                        callbacks=[es, rl],
                        verbose=1)

    # -------------------------
    # Predict on all sequences and align with original index
    # -------------------------
    preds_scaled = model.predict(X_all, batch_size=batch_size)
    preds = y_scaler.inverse_transform(preds_scaled).flatten()

    # create Predicted column with NaNs for first look_back rows
    df['Predicted'] = np.nan
    start_idx = look_back
    df.iloc[start_idx : start_idx + len(preds), df.columns.get_loc('Predicted')] = preds

    # round to 2 decimals
    for col in ['Average Price','Predicted','Lag_1','Lag_4','MA_4','MA_12']:
        if col in df.columns:
            df[col] = df[col].round(2)

    # -------------------------
    # Metrics (on rows where prediction exists)
    # -------------------------
    mask = ~np.isnan(df['Predicted'].values)
    y_true = df.loc[mask, 'Average Price'].values
    y_pred = df.loc[mask, 'Predicted'].values

    mae = round(mean_absolute_error(y_true, y_pred), 2)
    rmse = round(np.sqrt(mean_squared_error(y_true, y_pred)), 2)
    r2 = round(r2_score(y_true, y_pred), 4)
    mape = round(mean_absolute_percentage_error(y_true, y_pred), 2)
    accuracy = round(100 - mape, 2)

    metrics_list.append({
        'Crop': file.replace('.csv',''),
        'MAE': mae,
        'RMSE': rmse,
        'R2': r2,
        'MAPE(%)': mape,
        'Accuracy(%)': accuracy
    })

    print(f"✅ {file} | MAE={mae}, RMSE={rmse}, R²={r2}, MAPE={mape}%, Accuracy={accuracy}%")

    # -------------------------
    # Save model, CSV, graph
    # -------------------------
    model_save_path = os.path.join(output_models, file.replace('.csv','_tat_ha_weekly.h5'))
    model.save(model_save_path)

    out_csv = os.path.join(output_csv, file.replace('.csv','_tat_ha_weekly_updated.csv'))
    df_reset = df.reset_index()
    df_reset.to_csv(out_csv, index=False, float_format='%.2f')

    # graph
    plt.figure(figsize=(12,6))
    plt.plot(df_reset['Date'], df_reset['Average Price'], label='Actual', color='blue')
    plt.plot(df_reset['Date'], df_reset['Predicted'], label='Predicted (TAT+HA)', color='red', linestyle='dashed')
    plt.plot(df_reset['Date'], df_reset['MA_4'], label='MA_4', color='green', linestyle='--')
    plt.plot(df_reset['Date'], df_reset['MA_12'], label='MA_12', color='orange', linestyle='--')
    plt.title(f"TAT+HA Weekly Forecast - {file}")
    plt.xlabel("Date")
    plt.ylabel("Average Price")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(os.path.join(output_graphs, file.replace('.csv','_tat_ha_weekly_graph.png')))
    plt.close()

    # cleanup
    del model, X_all, X_train, X_val, y_all, y_train, y_val, preds_scaled, preds
    gc.collect()

# -----------------------------
# Save metrics summary (all crops)
# -----------------------------
metrics_df = pd.DataFrame(metrics_list).round(2)
metrics_df.to_csv(metrics_file, index=False)
print(f"\n📊 All crops processed — metrics saved to {metrics_file}")
print(metrics_df)


🚀 Processing Crop File: capsicum_2010_2024_master_weekly.csv
Epoch 1/60
21/21 ━━━━━━━━━━━━━━━━━━━━ 17s 132ms/step - loss: 0.4385 - mae: 0.4737 - val_loss: 0.0844 - val_mae: 0.2615 - learning_rate: 0.0010
Epoch 2/60
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 52ms/step - loss: 0.0466 - mae: 0.1673 - val_loss: 0.0576 - val_mae: 0.2044 - learning_rate: 0.0010
Epoch 3/60
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 44ms/step - loss: 0.0245 - mae: 0.1269 - val_loss: 0.0216 - val_mae: 0.1093 - learning_rate: 0.0010
Epoch 4/60
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 46ms/step - loss: 0.0209 - mae: 0.1138 - val_loss: 0.0301 - val_mae: 0.1331 - learning_rate: 0.0010
Epoch 5/60
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 47ms/step - loss: 0.0220 - mae: 0.1169 - val_loss: 0.0452 - val_mae: 0.1736 - learning_rate: 0.0010
Epoch 6/60
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 45ms/step - loss: 0.0207 - mae: 0.1118 - val_loss: 0.0401 - val_mae: 0.1605 - learning_rate: 0.0010
Epoch 7/60
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 53ms/step - loss: 0.0206 - mae: 0.1137 - val_loss

✅ capsicum_2010_2024_master_weekly.csv | MAE=474.51, RMSE=577.64, R²=0.3529, MAPE=21.33%, Accuracy=78.67%

🚀 Processing Crop File: onion_2010_2024_master_weekly.csv
Epoch 1/60
21/21 ━━━━━━━━━━━━━━━━━━━━ 24s 81ms/step - loss: 0.4091 - mae: 0.4575 - val_loss: 0.0544 - val_mae: 0.1698 - learning_rate: 0.0010
Epoch 2/60
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 47ms/step - loss: 0.0694 - mae: 0.2076 - val_loss: 0.0762 - val_mae: 0.2143 - learning_rate: 0.0010
Epoch 3/60
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 52ms/step - loss: 0.0319 - mae: 0.1347 - val_loss: 0.0444 - val_mae: 0.1532 - learning_rate: 0.0010
Epoch 4/60
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 50ms/step - loss: 0.0238 - mae: 0.1138 - val_loss: 0.0429 - val_mae: 0.1518 - learning_rate: 0.0010
Epoch 5/60
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 64ms/step - loss: 0.0192 - mae: 0.0991 - val_loss: 0.0392 - val_mae: 0.1481 - learning_rate: 0.0010
Epoch 6/60
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 46ms/step - loss: 0.0180 - mae: 0.0966 - val_loss: 0.0394 - val_mae: 0.1482 - learning_r

✅ onion_2010_2024_master_weekly.csv | MAE=360.92, RMSE=475.08, R²=0.4888, MAPE=22.79%, Accuracy=77.21%

🚀 Processing Crop File: tomato_2010_2024_master_weekly.csv
Epoch 1/60
21/21 ━━━━━━━━━━━━━━━━━━━━ 16s 69ms/step - loss: 0.4626 - mae: 0.4257 - val_loss: 0.0447 - val_mae: 0.1284 - learning_rate: 0.0010
Epoch 2/60
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step - loss: 0.0271 - mae: 0.1262 - val_loss: 0.0472 - val_mae: 0.1356 - learning_rate: 0.0010
Epoch 3/60
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step - loss: 0.0125 - mae: 0.0825 - val_loss: 0.0426 - val_mae: 0.1219 - learning_rate: 0.0010
Epoch 4/60
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 37ms/step - loss: 0.0088 - mae: 0.0684 - val_loss: 0.0379 - val_mae: 0.1123 - learning_rate: 0.0010
Epoch 5/60
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 37ms/step - loss: 0.0076 - mae: 0.0644 - val_loss: 0.0405 - val_mae: 0.1159 - learning_rate: 0.0010
Epoch 6/60
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 40ms/step - loss: 0.0078 - mae: 0.0639 - val_loss: 0.0360 - val_mae: 0.1115 - learning_rat

✅ tomato_2010_2024_master_weekly.csv | MAE=412.91, RMSE=622.99, R²=0.3527, MAPE=33.97%, Accuracy=66.03%

🚀 Processing Crop File: wheat_2010_2024_master_weekly.csv
Epoch 1/60
21/21 ━━━━━━━━━━━━━━━━━━━━ 27s 126ms/step - loss: 0.7012 - mae: 0.5934 - val_loss: 0.4389 - val_mae: 0.6581 - learning_rate: 0.0010
Epoch 2/60
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 39ms/step - loss: 0.0763 - mae: 0.2186 - val_loss: 0.1072 - val_mae: 0.3181 - learning_rate: 0.0010
Epoch 3/60
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 47ms/step - loss: 0.0223 - mae: 0.1174 - val_loss: 0.0708 - val_mae: 0.2562 - learning_rate: 0.0010
Epoch 4/60
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 38ms/step - loss: 0.0144 - mae: 0.0929 - val_loss: 0.0469 - val_mae: 0.2059 - learning_rate: 0.0010
Epoch 5/60
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - loss: 0.0092 - mae: 0.0758 - val_loss: 0.0710 - val_mae: 0.2587 - learning_rate: 0.0010
Epoch 6/60
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 39ms/step - loss: 0.0104 - mae: 0.0782 - val_loss: 0.0362 - val_mae: 0.1805 - learning_ra

✅ wheat_2010_2024_master_weekly.csv | MAE=122.12, RMSE=149.43, R²=0.9091, MAPE=5.41%, Accuracy=94.59%

📊 All crops processed — metrics saved to tat_ha_metrics_weekly.csv
                               Crop     MAE    RMSE    R2  MAPE(%)  \
0  capsicum_2010_2024_master_weekly  474.51  577.64  0.35    21.33   
1     onion_2010_2024_master_weekly  360.92  475.08  0.49    22.79   
2    tomato_2010_2024_master_weekly  412.91  622.99  0.35    33.97   
3     wheat_2010_2024_master_weekly  122.12  149.43  0.91     5.41   

   Accuracy(%)  
0        78.67  
1        77.21  
2        66.03  
3        94.59  


In [ ]:
#LSTM

In [9]:
import os
import gc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers, callbacks

# -----------------------------
# Paths / outputs
# -----------------------------
input_folder = "dataset"   # weekly CSVs
output_models = "lstm_output_models_weekly"
output_csv = "lstm_output_csv_weekly"
output_graphs = "lstm_output_graphs_weekly"
metrics_file = "lstm_metrics_weekly.csv"

os.makedirs(output_models, exist_ok=True)
os.makedirs(output_csv, exist_ok=True)
os.makedirs(output_graphs, exist_ok=True)

# -----------------------------
# Config (tweak if needed)
# -----------------------------
look_back = 30       # weeks of history used as input
batch_size = 32
epochs = 60
val_split = 0.15
lstm_units = 64
dropout_rate = 0.2
seed = 42

tf.random.set_seed(seed)
np.random.seed(seed)

# -----------------------------
# Helpers
# -----------------------------
def parse_dates_safe(s):
    try:
        return pd.to_datetime(s, dayfirst=True)
    except:
        return pd.to_datetime(s, dayfirst=False)

def mean_absolute_percentage_error(y_true, y_pred):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    mask = y_true != 0
    if mask.sum() == 0:
        return np.nan
    return np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100

def create_multivariate_dataset(y_scaled, exog_scaled, look_back):
    """
    Build multivariate sequences:
      y_scaled: (n,1), exog_scaled: (n,k)
    returns:
      X shape (n-look_back, look_back, 1+k), y shape (n-look_back,)
    """
    n = len(y_scaled)
    k = exog_scaled.shape[1] if exog_scaled is not None else 0
    X, Y = [], []
    for i in range(n - look_back):
        seq_y = y_scaled[i:i+look_back, 0]
        if k > 0:
            seq_exog = exog_scaled[i:i+look_back, :]
            seq = np.concatenate([seq_y.reshape(-1,1), seq_exog], axis=1)
        else:
            seq = seq_y.reshape(-1,1)
        X.append(seq)
        Y.append(y_scaled[i+look_back, 0])
    return np.array(X), np.array(Y)

# -----------------------------
# Build LSTM Model
# -----------------------------
def build_lstm_model(input_shape, units=64, dropout_rate=0.2, lr=0.001):
    """
    input_shape: (seq_len, num_features)
    returns compiled Keras model
    """
    model = models.Sequential()
    model.add(layers.Input(shape=input_shape))
    model.add(layers.LSTM(units, return_sequences=True))
    model.add(layers.Dropout(dropout_rate))
    model.add(layers.LSTM(units//2, return_sequences=False))
    model.add(layers.Dropout(dropout_rate))
    model.add(layers.Dense(units//2, activation='relu'))
    model.add(layers.Dense(1))
    model.compile(optimizer=optimizers.Adam(learning_rate=lr), loss='mse', metrics=['mae'])
    return model

# -----------------------------
# Main loop — process weekly crop files
# -----------------------------
metrics_list = []

for file in sorted(os.listdir(input_folder)):
    if not file.endswith(".csv"):
        continue

    print(f"\n🚀 Processing Crop File: {file}")
    file_path = os.path.join(input_folder, file)

    # -------------------------
    # Load & parse
    # -------------------------
    df = pd.read_csv(file_path)
    df['Date'] = parse_dates_safe(df['Date'])
    df = df.sort_values('Date')

    # aggregate duplicate dates numeric mean
    df = df.groupby('Date', as_index=False).mean(numeric_only=True)
    df.set_index('Date', inplace=True)

    # weekly frequency alignment
    df = df.asfreq('W')

    # check column
    if 'Average Price' not in df.columns:
        print(f"⚠ 'Average Price' not found in {file}, skipping.")
        continue

    # fill missing price values
    df['Average Price'] = df['Average Price'].ffill().bfill().fillna(df['Average Price'].mean())

    # -------------------------
    # Feature engineering (weekly)
    # -------------------------
    df['Lag_1'] = df['Average Price'].shift(1)
    df['Lag_4'] = df['Average Price'].shift(4)
    df['MA_4'] = df['Average Price'].rolling(window=4).mean()
    df['MA_12'] = df['Average Price'].rolling(window=12).mean()
    # fill feature NaNs
    df[['Lag_1','Lag_4','MA_4','MA_12']] = df[['Lag_1','Lag_4','MA_4','MA_12']].bfill().ffill()

    # -------------------------
    # Prepare arrays & scalers
    # -------------------------
    y = df[['Average Price']].values.astype('float32')
    exog_cols = ['Lag_1','Lag_4','MA_4','MA_12']
    exog = df[exog_cols].values.astype('float32')

    # scale
    y_scaler = MinMaxScaler()
    exog_scaler = MinMaxScaler()
    y_scaled = y_scaler.fit_transform(y)
    exog_scaled = exog_scaler.fit_transform(exog)

    # sequences
    X_all, y_all = create_multivariate_dataset(y_scaled, exog_scaled, look_back)
    if len(X_all) == 0:
        print(f"⚠ Not enough data after look_back={look_back} in {file}. Skipping.")
        continue

    # train/val split (chronological)
    val_size = int(len(X_all) * val_split)
    train_size = len(X_all) - val_size
    X_train, y_train = X_all[:train_size], y_all[:train_size]
    X_val, y_val = X_all[train_size:], y_all[train_size:]

    # -------------------------
    # Build model & train
    # -------------------------
    input_shape = (X_train.shape[1], X_train.shape[2])
    model = build_lstm_model(input_shape, units=lstm_units, dropout_rate=dropout_rate)

    es = callbacks.EarlyStopping(monitor='val_loss', patience=8, restore_best_weights=True)
    rl = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=4, min_lr=1e-6)

    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=epochs,
        batch_size=batch_size,
        callbacks=[es, rl],
        verbose=1
    )

    # -------------------------
    # Predict on all sequences and align with original index
    # -------------------------
    preds_scaled = model.predict(X_all, batch_size=batch_size)
    preds = y_scaler.inverse_transform(preds_scaled).flatten()

    # insert Predicted column with NaNs for first look_back rows
    df['Predicted'] = np.nan
    start_idx = look_back
    df.iloc[start_idx : start_idx + len(preds), df.columns.get_loc('Predicted')] = preds

    # round numeric columns to 2 decimals
    for col in ['Average Price','Predicted','Lag_1','Lag_4','MA_4','MA_12']:
        if col in df.columns:
            df[col] = df[col].round(2)

    # -------------------------
    # Metrics (on rows with predictions)
    # -------------------------
    mask = ~np.isnan(df['Predicted'].values)
    y_true = df.loc[mask, 'Average Price'].values
    y_pred = df.loc[mask, 'Predicted'].values

    mae = round(mean_absolute_error(y_true, y_pred), 2)
    rmse = round(np.sqrt(mean_squared_error(y_true, y_pred)), 2)
    r2 = round(r2_score(y_true, y_pred), 4)
    mape = round(mean_absolute_percentage_error(y_true, y_pred), 2)
    accuracy = round(100 - mape, 2)

    metrics_list.append({
        'Crop': file.replace('.csv',''),
        'MAE': mae,
        'RMSE': rmse,
        'R2': r2,
        'MAPE(%)': mape,
        'Accuracy(%)': accuracy
    })

    print(f"✅ {file} | MAE={mae}, RMSE={rmse}, R²={r2}, MAPE={mape}%, Accuracy={accuracy}%")

    # -------------------------
    # Save model, CSV, graph
    # -------------------------
    model_save_path = os.path.join(output_models, file.replace('.csv','_lstm_weekly.h5'))
    model.save(model_save_path)

    out_csv = os.path.join(output_csv, file.replace('.csv','_lstm_weekly_updated.csv'))
    df_reset = df.reset_index()
    df_reset.to_csv(out_csv, index=False, float_format='%.2f')

    plt.figure(figsize=(12,6))
    plt.plot(df_reset['Date'], df_reset['Average Price'], label='Actual', color='blue')
    plt.plot(df_reset['Date'], df_reset['Predicted'], label='Predicted (LSTM)', color='red', linestyle='dashed')
    plt.plot(df_reset['Date'], df_reset['MA_4'], label='MA_4', color='green', linestyle='--')
    plt.plot(df_reset['Date'], df_reset['MA_12'], label='MA_12', color='orange', linestyle='--')
    plt.title(f"LSTM Weekly Forecast - {file}")
    plt.xlabel("Date")
    plt.ylabel("Average Price")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(os.path.join(output_graphs, file.replace('.csv','_lstm_weekly_graph.png')))
    plt.close()

    # cleanup
    del model, X_all, X_train, X_val, y_all, y_train, y_val, preds_scaled, preds
    gc.collect()

# -----------------------------
# Save metrics summary (all crops)
# -----------------------------
metrics_df = pd.DataFrame(metrics_list).round(2)
metrics_df.to_csv(metrics_file, index=False)
print(f"\n📊 All crops processed — metrics saved to {metrics_file}")
print(metrics_df)


🚀 Processing Crop File: capsicum_2010_2024_master_weekly.csv
Epoch 1/60
21/21 ━━━━━━━━━━━━━━━━━━━━ 6s 50ms/step - loss: 0.0352 - mae: 0.1416 - val_loss: 0.0221 - val_mae: 0.1149 - learning_rate: 0.0010
Epoch 2/60
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - loss: 0.0142 - mae: 0.0932 - val_loss: 0.0302 - val_mae: 0.1398 - learning_rate: 0.0010
Epoch 3/60
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - loss: 0.0125 - mae: 0.0861 - val_loss: 0.0281 - val_mae: 0.1353 - learning_rate: 0.0010
Epoch 4/60
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - loss: 0.0112 - mae: 0.0820 - val_loss: 0.0273 - val_mae: 0.1336 - learning_rate: 0.0010
Epoch 5/60
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - loss: 0.0100 - mae: 0.0769 - val_loss: 0.0257 - val_mae: 0.1287 - learning_rate: 0.0010
Epoch 6/60
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.0090 - mae: 0.0728 - val_loss: 0.0233 - val_mae: 0.1212 - learning_rate: 5.0000e-04
Epoch 7/60
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 0.0086 - mae: 0.0709 - val_lo

✅ capsicum_2010_2024_master_weekly.csv | MAE=245.6, RMSE=333.03, R²=0.7849, MAPE=9.5%, Accuracy=90.5%

🚀 Processing Crop File: onion_2010_2024_master_weekly.csv
Epoch 1/60
21/21 ━━━━━━━━━━━━━━━━━━━━ 12s 103ms/step - loss: 0.0296 - mae: 0.1224 - val_loss: 0.0487 - val_mae: 0.1675 - learning_rate: 0.0010
Epoch 2/60
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - loss: 0.0131 - mae: 0.0787 - val_loss: 0.0307 - val_mae: 0.1320 - learning_rate: 0.0010
Epoch 3/60
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - loss: 0.0102 - mae: 0.0683 - val_loss: 0.0178 - val_mae: 0.1051 - learning_rate: 0.0010
Epoch 4/60
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - loss: 0.0088 - mae: 0.0619 - val_loss: 0.0141 - val_mae: 0.0933 - learning_rate: 0.0010
Epoch 5/60
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step - loss: 0.0072 - mae: 0.0558 - val_loss: 0.0099 - val_mae: 0.0773 - learning_rate: 0.0010
Epoch 6/60
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - loss: 0.0073 - mae: 0.0571 - val_loss: 0.0132 - val_mae: 0.0874 - learning_rate

✅ onion_2010_2024_master_weekly.csv | MAE=125.15, RMSE=201.3, R²=0.9082, MAPE=7.06%, Accuracy=92.94%

🚀 Processing Crop File: tomato_2010_2024_master_weekly.csv
Epoch 1/60
21/21 ━━━━━━━━━━━━━━━━━━━━ 12s 84ms/step - loss: 0.0093 - mae: 0.0643 - val_loss: 0.0361 - val_mae: 0.1170 - learning_rate: 0.0010
Epoch 2/60
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - loss: 0.0062 - mae: 0.0539 - val_loss: 0.0328 - val_mae: 0.1122 - learning_rate: 0.0010
Epoch 3/60
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - loss: 0.0055 - mae: 0.0518 - val_loss: 0.0301 - val_mae: 0.1092 - learning_rate: 0.0010
Epoch 4/60
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - loss: 0.0049 - mae: 0.0486 - val_loss: 0.0270 - val_mae: 0.1063 - learning_rate: 0.0010
Epoch 5/60
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - loss: 0.0042 - mae: 0.0449 - val_loss: 0.0238 - val_mae: 0.1044 - learning_rate: 0.0010
Epoch 6/60
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - loss: 0.0038 - mae: 0.0436 - val_loss: 0.0202 - val_mae: 0.0939 - learning_rate:

✅ tomato_2010_2024_master_weekly.csv | MAE=126.31, RMSE=200.42, R²=0.933, MAPE=9.42%, Accuracy=90.58%

🚀 Processing Crop File: wheat_2010_2024_master_weekly.csv
Epoch 1/60
21/21 ━━━━━━━━━━━━━━━━━━━━ 10s 76ms/step - loss: 0.0222 - mae: 0.1000 - val_loss: 0.0254 - val_mae: 0.1514 - learning_rate: 0.0010
Epoch 2/60
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step - loss: 0.0028 - mae: 0.0401 - val_loss: 0.0142 - val_mae: 0.1111 - learning_rate: 0.0010
Epoch 3/60
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - loss: 0.0023 - mae: 0.0346 - val_loss: 0.0179 - val_mae: 0.1267 - learning_rate: 0.0010
Epoch 4/60
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - loss: 0.0021 - mae: 0.0337 - val_loss: 0.0202 - val_mae: 0.1353 - learning_rate: 0.0010
Epoch 5/60
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - loss: 0.0024 - mae: 0.0350 - val_loss: 0.0145 - val_mae: 0.1121 - learning_rate: 0.0010
Epoch 6/60
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - loss: 0.0020 - mae: 0.0322 - val_loss: 0.0229 - val_mae: 0.1445 - learning_rate:

✅ wheat_2010_2024_master_weekly.csv | MAE=84.73, RMSE=119.03, R²=0.9423, MAPE=3.5%, Accuracy=96.5%

📊 All crops processed — metrics saved to lstm_metrics_weekly.csv
                               Crop     MAE    RMSE    R2  MAPE(%)  \
0  capsicum_2010_2024_master_weekly  245.60  333.03  0.78     9.50   
1     onion_2010_2024_master_weekly  125.15  201.30  0.91     7.06   
2    tomato_2010_2024_master_weekly  126.31  200.42  0.93     9.42   
3     wheat_2010_2024_master_weekly   84.73  119.03  0.94     3.50   

   Accuracy(%)  
0        90.50  
1        92.94  
2        90.58  
3        96.50  


In [ ]:
#GRU

In [10]:
import os
import gc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers, callbacks

# -----------------------------
# Paths / Outputs
# -----------------------------
input_folder = "dataset"  # folder containing weekly CSVs
output_models = "gru_transformer_output_models_weekly"
output_csv = "gru_transformer_output_csv_weekly"
output_graphs = "gru_transformer_output_graphs_weekly"
metrics_file = "gru_transformer_metrics_weekly.csv"

os.makedirs(output_models, exist_ok=True)
os.makedirs(output_csv, exist_ok=True)
os.makedirs(output_graphs, exist_ok=True)

# -----------------------------
# Config
# -----------------------------
look_back = 30       # number of weeks used for training window
batch_size = 32
epochs = 60
val_split = 0.15
gru_units = 64
ff_dim = 128
dropout_rate = 0.2
learning_rate = 0.001
seed = 42

tf.random.set_seed(seed)
np.random.seed(seed)

# -----------------------------
# Helper Functions
# -----------------------------
def parse_dates_safe(series):
    try:
        return pd.to_datetime(series, dayfirst=True)
    except:
        return pd.to_datetime(series, dayfirst=False)

def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    mask = y_true != 0
    return np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100 if np.any(mask) else np.nan

def create_multivariate_dataset(y_scaled, exog_scaled, look_back):
    n = len(y_scaled)
    k = exog_scaled.shape[1] if exog_scaled is not None else 0
    X, Y = [], []
    for i in range(n - look_back):
        seq_y = y_scaled[i:i+look_back, 0]
        if k > 0:
            seq_exog = exog_scaled[i:i+look_back, :]
            seq = np.concatenate([seq_y.reshape(-1,1), seq_exog], axis=1)
        else:
            seq = seq_y.reshape(-1,1)
        X.append(seq)
        Y.append(y_scaled[i+look_back, 0])
    return np.array(X), np.array(Y)

def positional_encoding(seq_len, d_model):
    pos = np.arange(seq_len)[:, np.newaxis]
    i = np.arange(d_model)[np.newaxis, :]
    angle_rates = 1 / np.power(10000, (2 * (i//2)) / np.float32(d_model))
    angle_rads = pos * angle_rates
    pe = np.zeros_like(angle_rads)
    pe[:, 0::2] = np.sin(angle_rads[:, 0::2])
    pe[:, 1::2] = np.cos(angle_rads[:, 1::2])
    return tf.cast(pe, tf.float32)

# -----------------------------
# Build GRU Transformer Model
# -----------------------------
def build_gru_transformer(input_shape, gru_units=64, ff_dim=128, dropout_rate=0.2, lr=0.001):
    seq_len, num_feat = input_shape
    inp = layers.Input(shape=(seq_len, num_feat))

    # Step 1: GRU Layers
    x = layers.GRU(gru_units, return_sequences=True)(inp)
    x = layers.Dropout(dropout_rate)(x)
    x = layers.GRU(gru_units//2, return_sequences=True)(x)
    x = layers.LayerNormalization(epsilon=1e-6)(x)

    # Step 2: Positional Encoding + Feedforward Transformer block
    pe = positional_encoding(seq_len, gru_units//2)
    x = x + pe

    # Transformer-style FFN block
    ff = layers.Dense(ff_dim, activation='relu')(x)
    ff = layers.Dense(gru_units//2)(ff)
    ff = layers.Dropout(dropout_rate)(ff)
    x = layers.LayerNormalization(epsilon=1e-6)(x + ff)

    # Step 3: Global Pooling + Dense Output
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dense(ff_dim//2, activation='relu')(x)
    x = layers.Dropout(dropout_rate)(x)
    out = layers.Dense(1)(x)

    model = models.Model(inputs=inp, outputs=out)
    model.compile(optimizer=optimizers.Adam(learning_rate=lr), loss='mse', metrics=['mae'])
    return model

# -----------------------------
# Main loop — process all crop CSVs
# -----------------------------
metrics_list = []

for file in sorted(os.listdir(input_folder)):
    if not file.endswith(".csv"):
        continue

    print(f"\n🚀 Processing Crop File: {file}")
    file_path = os.path.join(input_folder, file)

    # Load dataset
    df = pd.read_csv(file_path)
    df['Date'] = parse_dates_safe(df['Date'])
    df = df.sort_values('Date')

    # Aggregate duplicate dates (mean)
    df = df.groupby('Date', as_index=False).mean(numeric_only=True)
    df.set_index('Date', inplace=True)

    # Align weekly frequency
    df = df.asfreq('W')

    # Handle missing prices
    if 'Average Price' not in df.columns:
        print(f"⚠ Skipping {file}: No 'Average Price' column.")
        continue
    df['Average Price'] = df['Average Price'].ffill().bfill().fillna(df['Average Price'].mean())

    # Feature Engineering
    df['Lag_1'] = df['Average Price'].shift(1)
    df['Lag_4'] = df['Average Price'].shift(4)
    df['MA_4'] = df['Average Price'].rolling(window=4).mean()
    df['MA_12'] = df['Average Price'].rolling(window=12).mean()
    df[['Lag_1','Lag_4','MA_4','MA_12']] = df[['Lag_1','Lag_4','MA_4','MA_12']].bfill().ffill()

    # Prepare scaled data
    y = df[['Average Price']].values.astype('float32')
    exog_cols = ['Lag_1','Lag_4','MA_4','MA_12']
    exog = df[exog_cols].values.astype('float32')

    y_scaler = MinMaxScaler()
    exog_scaler = MinMaxScaler()
    y_scaled = y_scaler.fit_transform(y)
    exog_scaled = exog_scaler.fit_transform(exog)

    X_all, y_all = create_multivariate_dataset(y_scaled, exog_scaled, look_back)
    if len(X_all) == 0:
        print(f"⚠ Not enough data for look_back={look_back}. Skipping {file}.")
        continue

    # Train/Val Split
    val_size = int(len(X_all) * val_split)
    train_size = len(X_all) - val_size
    X_train, y_train = X_all[:train_size], y_all[:train_size]
    X_val, y_val = X_all[train_size:], y_all[train_size:]

    # Build and Train model
    input_shape = (X_train.shape[1], X_train.shape[2])
    model = build_gru_transformer(input_shape, gru_units=gru_units, ff_dim=ff_dim, dropout_rate=dropout_rate)

    es = callbacks.EarlyStopping(monitor='val_loss', patience=8, restore_best_weights=True)
    rl = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=4, min_lr=1e-6)

    history = model.fit(X_train, y_train,
                        validation_data=(X_val, y_val),
                        epochs=epochs,
                        batch_size=batch_size,
                        callbacks=[es, rl],
                        verbose=1)

    # Predictions
    preds_scaled = model.predict(X_all, batch_size=batch_size)
    preds = y_scaler.inverse_transform(preds_scaled).flatten()

    df['Predicted'] = np.nan
    start_idx = look_back
    df.iloc[start_idx:start_idx+len(preds), df.columns.get_loc('Predicted')] = preds

    # Round to 2 decimals
    for col in ['Average Price','Predicted','Lag_1','Lag_4','MA_4','MA_12']:
        if col in df.columns:
            df[col] = df[col].round(2)

    # Compute Metrics
    mask = ~np.isnan(df['Predicted'].values)
    y_true = df.loc[mask, 'Average Price'].values
    y_pred = df.loc[mask, 'Predicted'].values

    mae = round(mean_absolute_error(y_true, y_pred), 2)
    rmse = round(np.sqrt(mean_squared_error(y_true, y_pred)), 2)
    r2 = round(r2_score(y_true, y_pred), 4)
    mape = round(mean_absolute_percentage_error(y_true, y_pred), 2)
    accuracy = round(100 - mape, 2)

    metrics_list.append({
        'Crop': file.replace('.csv',''),
        'MAE': mae,
        'RMSE': rmse,
        'R2': r2,
        'MAPE(%)': mape,
        'Accuracy(%)': accuracy
    })

    print(f"✅ {file} | MAE={mae}, RMSE={rmse}, R²={r2}, MAPE={mape}%, Accuracy={accuracy}%")

    # Save model, CSV, and graph
    model.save(os.path.join(output_models, file.replace('.csv','_gru_transformer_weekly.h5')))

    out_csv = os.path.join(output_csv, file.replace('.csv','_gru_transformer_weekly_updated.csv'))
    df_reset = df.reset_index()
    df_reset.to_csv(out_csv, index=False, float_format='%.2f')

    plt.figure(figsize=(12,6))
    plt.plot(df_reset['Date'], df_reset['Average Price'], label='Actual', color='blue')
    plt.plot(df_reset['Date'], df_reset['Predicted'], label='Predicted (GRU Transformer)', color='red', linestyle='dashed')
    plt.plot(df_reset['Date'], df_reset['MA_4'], label='MA_4', color='green', linestyle='--')
    plt.plot(df_reset['Date'], df_reset['MA_12'], label='MA_12', color='orange', linestyle='--')
    plt.title(f"GRU Transformer Weekly Forecast - {file}")
    plt.xlabel("Date")
    plt.ylabel("Average Price")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(os.path.join(output_graphs, file.replace('.csv','_gru_transformer_weekly_graph.png')))
    plt.close()

    del model, X_all, X_train, X_val, y_all, y_train, y_val, preds_scaled, preds
    gc.collect()

# -----------------------------
# Save combined metrics CSV
# -----------------------------
metrics_df = pd.DataFrame(metrics_list).round(2)
metrics_df.to_csv(metrics_file, index=False)
print(f"\n📊 All crops processed — metrics saved to {metrics_file}")
print(metrics_df)


🚀 Processing Crop File: capsicum_2010_2024_master_weekly.csv
Epoch 1/60
21/21 ━━━━━━━━━━━━━━━━━━━━ 26s 173ms/step - loss: 0.2820 - mae: 0.3763 - val_loss: 0.0511 - val_mae: 0.1876 - learning_rate: 0.0010
Epoch 2/60
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 49ms/step - loss: 0.0654 - mae: 0.2075 - val_loss: 0.0329 - val_mae: 0.1389 - learning_rate: 0.0010
Epoch 3/60
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 35ms/step - loss: 0.0422 - mae: 0.1625 - val_loss: 0.0397 - val_mae: 0.1578 - learning_rate: 0.0010
Epoch 4/60
21/21 ━━━━━━━━━━━━━━━━━━━━ 2s 46ms/step - loss: 0.0287 - mae: 0.1354 - val_loss: 0.0626 - val_mae: 0.2160 - learning_rate: 0.0010
Epoch 5/60
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 44ms/step - loss: 0.0272 - mae: 0.1319 - val_loss: 0.0541 - val_mae: 0.1958 - learning_rate: 0.0010
Epoch 6/60
21/21 ━━━━━━━━━━━━━━━━━━━━ 2s 66ms/step - loss: 0.0242 - mae: 0.1235 - val_loss: 0.0442 - val_mae: 0.1709 - learning_rate: 0.0010
Epoch 7/60
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 60ms/step - loss: 0.0217 - mae: 0.1180 - val_loss

✅ capsicum_2010_2024_master_weekly.csv | MAE=545.96, RMSE=655.26, R²=0.1673, MAPE=25.26%, Accuracy=74.74%

🚀 Processing Crop File: onion_2010_2024_master_weekly.csv
Epoch 1/60
21/21 ━━━━━━━━━━━━━━━━━━━━ 16s 183ms/step - loss: 0.1048 - mae: 0.2503 - val_loss: 0.0409 - val_mae: 0.1515 - learning_rate: 0.0010
Epoch 2/60
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 38ms/step - loss: 0.0298 - mae: 0.1289 - val_loss: 0.0629 - val_mae: 0.1878 - learning_rate: 0.0010
Epoch 3/60
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step - loss: 0.0201 - mae: 0.1057 - val_loss: 0.0575 - val_mae: 0.1782 - learning_rate: 0.0010
Epoch 4/60
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 49ms/step - loss: 0.0183 - mae: 0.0992 - val_loss: 0.0440 - val_mae: 0.1535 - learning_rate: 0.0010
Epoch 5/60
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 40ms/step - loss: 0.0171 - mae: 0.0959 - val_loss: 0.0491 - val_mae: 0.1632 - learning_rate: 0.0010
Epoch 6/60
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 44ms/step - loss: 0.0156 - mae: 0.0894 - val_loss: 0.0403 - val_mae: 0.1477 - learning_

✅ onion_2010_2024_master_weekly.csv | MAE=306.61, RMSE=380.94, R²=0.6713, MAPE=19.93%, Accuracy=80.07%

🚀 Processing Crop File: tomato_2010_2024_master_weekly.csv
Epoch 1/60
21/21 ━━━━━━━━━━━━━━━━━━━━ 21s 141ms/step - loss: 0.0908 - mae: 0.2169 - val_loss: 0.0359 - val_mae: 0.1140 - learning_rate: 0.0010
Epoch 2/60
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 43ms/step - loss: 0.0138 - mae: 0.0893 - val_loss: 0.0431 - val_mae: 0.1251 - learning_rate: 0.0010
Epoch 3/60
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 43ms/step - loss: 0.0105 - mae: 0.0769 - val_loss: 0.0414 - val_mae: 0.1195 - learning_rate: 0.0010
Epoch 4/60
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 41ms/step - loss: 0.0087 - mae: 0.0681 - val_loss: 0.0400 - val_mae: 0.1171 - learning_rate: 0.0010
Epoch 5/60
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 41ms/step - loss: 0.0087 - mae: 0.0688 - val_loss: 0.0346 - val_mae: 0.1091 - learning_rate: 0.0010
Epoch 6/60
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 35ms/step - loss: 0.0085 - mae: 0.0682 - val_loss: 0.0370 - val_mae: 0.1115 - learning_ra

✅ tomato_2010_2024_master_weekly.csv | MAE=245.38, RMSE=413.59, R²=0.7147, MAPE=18.13%, Accuracy=81.87%

🚀 Processing Crop File: wheat_2010_2024_master_weekly.csv
Epoch 1/60
21/21 ━━━━━━━━━━━━━━━━━━━━ 17s 137ms/step - loss: 0.1433 - mae: 0.2872 - val_loss: 0.0357 - val_mae: 0.1741 - learning_rate: 0.0010
Epoch 2/60
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 41ms/step - loss: 0.0204 - mae: 0.1126 - val_loss: 0.0452 - val_mae: 0.1981 - learning_rate: 0.0010
Epoch 3/60
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 42ms/step - loss: 0.0129 - mae: 0.0919 - val_loss: 0.0970 - val_mae: 0.3028 - learning_rate: 0.0010
Epoch 4/60
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 39ms/step - loss: 0.0104 - mae: 0.0818 - val_loss: 0.0697 - val_mae: 0.2543 - learning_rate: 0.0010
Epoch 5/60
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 50ms/step - loss: 0.0083 - mae: 0.0698 - val_loss: 0.0385 - val_mae: 0.1845 - learning_rate: 0.0010
Epoch 6/60
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 50ms/step - loss: 0.0057 - mae: 0.0590 - val_loss: 0.0419 - val_mae: 0.1940 - learning_ra

✅ wheat_2010_2024_master_weekly.csv | MAE=97.85, RMSE=134.53, R²=0.9263, MAPE=4.19%, Accuracy=95.81%

📊 All crops processed — metrics saved to gru_transformer_metrics_weekly.csv
                               Crop     MAE    RMSE    R2  MAPE(%)  \
0  capsicum_2010_2024_master_weekly  545.96  655.26  0.17    25.26   
1     onion_2010_2024_master_weekly  306.61  380.94  0.67    19.93   
2    tomato_2010_2024_master_weekly  245.38  413.59  0.71    18.13   
3     wheat_2010_2024_master_weekly   97.85  134.53  0.93     4.19   

   Accuracy(%)  
0        74.74  
1        80.07  
2        81.87  
3        95.81  
